<a href="https://colab.research.google.com/github/matthew110395/12004210_DataAnalytics/blob/main/12004210_DAOTW_Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introduction
This assignment builds on the New York taxi problem identified in assignment 1, where the City of New York is looking for a way to accurately predict the number of collisions on a particular day of the week. Within this document two different types of machine learning models will be utilised to predict the number of collisions. The linear relationships identified in assignment 1 will be used to create linear regression models. Deep Learning Neural Network (DNN) models will be used to predict the number of collisions where the relationship is not linear. 

#Imports
To prepare data for machine learning the pandas package has been used, alongside the numpy package which has been used to aid with mathematical functions.

As within part 1 of this assignment, the data file containing location data exceeds the size limit for hosting within github. To overcome this the file was zipped. To extract the data the zipfile package has been used.

Within this document, TensorFlow is used for machine learning, with both linear regression models and a Deep Neural Network models. TensorFlow version 1 is unsupported within Google Colab, therefore must be installed using a package manager.

Shutil is also imported to allow for file management, in particular the removal of saved models.

In [1]:
#Import Packages
import pandas as pd
import numpy as np
import zipfile
!pip install tensorflow==1.15.2
import tensorflow as tf
import shutil  

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 110.5 MB 30 kB/s 
     |████████████████████████████████| 3.8 MB 24.0 MB/s 
     |████████████████████████████████| 50 kB 2.4 MB/s 
     |████████████████████████████████| 503 kB 47.6 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=5e84397cd1cae56800e51dec65b16abafaf0e28c7b2e15129e0365fb1b615ee8
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting un

#Linear Regressor
Throughout assignment 1 a number of linear relationships were uncovered within the dataset. These relationships form the basis of the linear regression models below.

A linear regressor is used to predict an output variable based on one or more input variables (IBM n.d.).

To improve the accuracy of the model the target values are scaled. This reduces the range of collisions from 188-1161 to 0.1619... - 1 which allows for quicker training  (Zhang 2019).

In [2]:
#Scale to maximum number of collisions
SCALE_COLLISIONS=1161

##Precipitation
As uncovered in assignment 1; as the volume of precipitation increases, the number of collisions increase. 

The datafile produced in assignment is imported.

In [3]:
#Read File
df_prcp = pd.read_csv('https://raw.githubusercontent.com/matthew110395/12004210_DataAnalytics/main/prcp_clean.csv', index_col=0, )
print(df_prcp[:6])

   day  year  mo  da collision_date  NUM_COLLISIONS  temp  dewp     slp  \
1    5  2020   1  24     2020-01-24             524  37.3  33.7  1028.5   
2    2  2021   1  12     2021-01-12             278  37.0  29.1  1019.0   
3    5  2021   1  22     2021-01-22             254  36.5  28.4  1003.1   
4    3  2021   1  27     2021-01-27             262  34.6  33.8  1012.8   
5    2  2021   1  26     2021-01-26             263  31.9  23.4  1016.9   
6    1  2022   1  24     2022-01-24             237  34.5  23.8  1010.6   

   visib  ...   max   min  prcp   sndp  fog  rain_drizzle  snow_ice_pellets  \
1    6.5  ...  46.0  19.9  0.00  999.9    1             0                 0   
2   10.0  ...  44.1  21.0  0.00  999.9    0             0                 0   
3   10.0  ...  44.1  19.9  0.00  999.9    0             0                 0   
4    8.0  ...  41.0  28.9  0.25  999.9    1             1                 0   
5    9.0  ...  37.9  21.0  0.00  999.9    1             0                 0   


In order to create the linear regression model, extra columns are removed to simplify the model with the aim of reducing error values.

The incomplete years (2012 and 2022) are removed, along with the erroneous data for 2020 and 2021.

To aid with the production of the model the target is moved to the end of the data table.

In [4]:
#Remove Cols not Required 
df_prcp = df_prcp.drop(columns=['collision_date', 'temp', 'dewp', 'slp','visib','max','min','sndp','wdsp','mxpsd','gust','thunder','tornado_funnel_cloud'])
df_prcp = df_prcp.loc[df_prcp["year"] != 2012]
df_prcp = df_prcp.loc[df_prcp["year"] < 2020]
cols = df_prcp['NUM_COLLISIONS']
df_prcp = df_prcp.drop(columns=['NUM_COLLISIONS'])
#Move NUM_COLLISIONS to end
df_prcp.insert(loc=9, column='NUM_COLLISIONS', value=cols)
print(df_prcp[:6])
df_prcp.describe()

    day  year  mo  da  prcp  fog  rain_drizzle  snow_ice_pellets  hail  \
49    4  2016   1  28  0.09    0             0                 0     0   
51    5  2014   1  17  0.00    1             0                 0     0   
54    1  2016   1  25  0.02    0             0                 0     0   
55    5  2016   1  29  0.00    0             0                 0     0   
58    5  2017   1  20  0.00    0             0                 0     0   
59    7  2013   1  13  0.01    1             0                 0     0   

    NUM_COLLISIONS  
49             681  
51             589  
54             658  
55             645  
58             605  
59             373  


,day,year,mo,da,prcp,fog,rain_drizzle,snow_ice_pellets,hail,NUM_COLLISIONS
count,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000
mean,3.998425,2015.989366,6.518708,15.745569,0.122588,0.253249,0.375345,0.085467,0.000394,599.135093
std,2.003542,1.996126,3.455211,8.803199,0.329143,0.434958,0.484307,0.279630,0.019846,100.299164
min,1.000000,2013.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,188.000000
25%,2.000000,2014.000000,4.000000,8.000000,0.000000,0.000000,0.000000,0.000000,0.000000,531.000000
50%,4.000000,2016.000000,7.000000,16.000000,0.000000,0.000000,0.000000,0.000000,0.000000,602.000000
75%,6.000000,2018.000000,10.000000,23.000000,0.060000,1.000000,1.000000,0.000000,0.000000,665.000000
max,7.000000,2019.000000,12.000000,31.000000,3.760000,1.000000,1.000000,1.000000,1.000000,1161.000000


To remove any bias within the dataset, it is randomly shuffled. The data is then split into the predictors and the target.

In [5]:
# Shuffle the data
shuffle = df_prcp.iloc[np.random.permutation(len(df_prcp))]

# Select all apart from last col
predictors = shuffle.iloc[:,0:-1]
print(predictors[:6])
print(shuffle[:6])

      day  year  mo  da  prcp  fog  rain_drizzle  snow_ice_pellets  hail
2384    4  2013   8  15  0.00    0             0                 0     0
1756    1  2019   6   3  0.00    0             0                 0     0
519     6  2014   2  22  0.44    1             1                 0     0
3229    5  2013  11  22  0.00    0             1                 0     0
2823    4  2016  10  13  0.00    0             0                 0     0
2559    6  2015   9  19  0.01    1             1                 0     0
      day  year  mo  da  prcp  fog  rain_drizzle  snow_ice_pellets  hail  \
2384    4  2013   8  15  0.00    0             0                 0     0   
1756    1  2019   6   3  0.00    0             0                 0     0   
519     6  2014   2  22  0.44    1             1                 0     0   
3229    5  2013  11  22  0.00    0             1                 0     0   
2823    4  2016  10  13  0.00    0             0                 0     0   
2559    6  2015   9  19  0.01    

In [6]:
# Select Target (last col)
targets = shuffle.iloc[:,-1]

print(targets[:6])

2384    599
1756    659
519     587
3229    702
2823    757
2559    655
Name: NUM_COLLISIONS, dtype: int64


In [7]:
#Split to test and train data
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
print(trainsize)
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize
print(testsize)
nppredictors = 9
noutputs = 1

2031
508


In [8]:
# logging for tensorflow
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

# removes a saved model from the last training attempt.
shutil.rmtree('/tmp/linear_regression_trained_model_prcp', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_prcp', optimizer=tf.train.AdamOptimizer(learning_rate=0.00001), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))
print("starting to train");
#Train Model
estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_COLLISIONS, steps=10000)
preds = estimator.predict(x=predictors[trainsize:].values)
predslistscale = preds['scores']*SCALE_COLLISIONS

pred = format(str(predslistscale))
#Test Model
rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('LinearRegression has RMSE of {0}'.format(rmse));

avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])
rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please specify feature columns explicitly.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please feed input to tf.data to support dask.
Instructions for updating:
Please access pandas data directly.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please convert numpy dtypes explicitly.
Instructions for updating:
Please specify feature columns explicitly.
Instructions for updating:
Please switch to tf.contrib.estimator.*_head.
Instructions for updating:
Please replace uses of any Estimator from tf.contri

starting to train


Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.EstimatorSpec. You can use the `estimator_spec` method to create an equivalent one.
INFO:tensorflow:Create CheckpointSaverHook.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/linear_regression_trained_model_prcp/model.ckpt.
INFO:tensorflow:loss = 0.2818297, step = 1
INFO:tensorflow:global_step/sec: 780.763
INFO:tensorflow:loss = 0.008050647, step = 101 (0.133 sec)
INFO:tensorflow:global_step/sec: 739.456
INFO:tensorflow:loss = 0.0068118866, step = 201 (0.135 sec)
INFO:tensorflow:global_step/sec: 831.683
INFO:tensorflow:loss = 0.0067442693, step = 301 (0.118 sec)
INFO:tensorflow:global_step/sec: 885.818
INFO:tensorflow:loss = 0.0077139656, step = 401 (0.113 sec)
INFO:tensorflow:global_ste

LinearRegression has RMSE of 94.04293216822417
Just using average = 597.4963072378139 has RMSE of 100.75322354041994


A number of learning rates were used to determine a suitable learning rate for the model. As the learning rate decreases the overall time to train the dataset increases (Zulkifli 2018).

In [9]:
#Check Error Rate
print(len(predictors[trainsize:].values))
print(len(targets[trainsize:].values.reshape(testsize, noutputs)/SCALE_COLLISIONS))
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_prcp', enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors[trainsize:].values)))
preds = estimator.predict(x=predictors[trainsize:].values)

print(preds['scores'])
print(targets[trainsize:].values/SCALE_COLLISIONS)

testdf = pd.DataFrame.from_dict(data={
    'pred': preds['scores'],
    'actual': targets[trainsize:].values/SCALE_COLLISIONS
})

testdf['diff'] = testdf['actual'] - testdf['pred']

error=testdf['diff'].mean()*SCALE_COLLISIONS
avgcol=targets[trainsize:].mean()
print('The trained model has an aproximate error rate of {0} which equates to {1}%'.format(error, round((error/avgcol)*100),1));


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f698b7c4c10>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_prcp', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/linear_regression_trained_model_prc

508
508
[0.49244517 0.48378417 0.506735   0.5241162  0.49296534 0.515598
 0.5320558  0.5080096  0.56285214 0.46699926 0.54114205 0.55445117
 0.52352196 0.47925076 0.55024    0.5189467  0.53738964 0.5017981
 0.48919103 0.51320815 0.48409644 0.46274322 0.5437057  0.48423973
 0.5334265  0.5465016  0.47476095 0.47122625 0.511074   0.5185038
 0.48082098 0.5352506  0.5126752  0.54007363 0.5392867  0.5332217
 0.55164385 0.5502804  0.54471076 0.5083329  0.51693255 0.49409437
 0.52621365 0.5461434  0.52084565 0.5519722  0.50297165 0.56764555
 0.5146872  0.49352923 0.51807636 0.4931414  0.5235054  0.4998226
 0.51515347 0.55737334 0.49201348 0.46954829 0.49867424 0.55823106
 0.51544356 0.5282449  0.534032   0.45085797 0.50553536 0.521301
 0.56145453 0.54818887 0.50697756 0.560859   0.5362219  0.51866055
 0.4883773  0.46688813 0.5504946  0.485053   0.5291783  0.49491972
 0.51075333 0.5039496  0.5533774  0.53664774 0.56532747 0.518715
 0.48536047 0.5126382  0.5126378  0.5621793  0.46768704 0.521689

Two main tests have been applied to the model. The Route Mean Squared Error (RMSE) and a comparison between the target values in the testing dataset and the predicted values using the predictors in the testing dataset.

Predominantly the RMSE of the model is lower than that of the average. This indicates that the model makes more accurate predictions compared to the average.

##Dew Point (dewp)
A relationship between dew point and the number of collisions was also uncovered in assignment 1. This linear relationship suggests that as the dew point increases the number of collisions increase. 

The process to produce the model follows the same process as the precipitation model.

In [10]:
#Read Data
df_dewp = pd.read_csv('https://raw.githubusercontent.com/matthew110395/12004210_DataAnalytics/main/dewp_clean.csv', index_col=0, )
print(df_dewp[:6])

   day  year  mo  da collision_date  NUM_COLLISIONS  temp  dewp     slp  \
1    5  2020   1  24     2020-01-24             524  37.3  33.7  1028.5   
2    2  2021   1  12     2021-01-12             278  37.0  29.1  1019.0   
3    5  2021   1  22     2021-01-22             254  36.5  28.4  1003.1   
4    3  2021   1  27     2021-01-27             262  34.6  33.8  1012.8   
5    2  2021   1  26     2021-01-26             263  31.9  23.4  1016.9   
6    1  2022   1  24     2022-01-24             237  34.5  23.8  1010.6   

   visib  ...   max   min  prcp   sndp  fog  rain_drizzle  snow_ice_pellets  \
1    6.5  ...  46.0  19.9  0.00  999.9    1             0                 0   
2   10.0  ...  44.1  21.0  0.00  999.9    0             0                 0   
3   10.0  ...  44.1  19.9  0.00  999.9    0             0                 0   
4    8.0  ...  41.0  28.9  0.25  999.9    1             1                 0   
5    9.0  ...  37.9  21.0  0.00  999.9    1             0                 0   


In [11]:
#Remove Cols not Required 
df_dewp = df_dewp.drop(columns=['collision_date', 'temp', 'prcp', 'slp','visib','max','min','sndp','wdsp','mxpsd','gust','thunder','tornado_funnel_cloud','fog','rain_drizzle','snow_ice_pellets','hail'])
df_dewp = df_dewp.loc[df_dewp["year"] != 2012]
df_dewp = df_dewp.loc[df_dewp["year"] < 2020]
cols = df_dewp['NUM_COLLISIONS']
df_dewp = df_dewp.drop(columns=['NUM_COLLISIONS'])
#Move target to end
df_dewp.insert(loc=5, column='NUM_COLLISIONS', value=cols)
print(df_dewp[:6])
df_dewp.describe()

    day  year  mo  da  dewp  NUM_COLLISIONS
49    4  2016   1  28  24.4             681
51    5  2014   1  17  35.8             589
54    1  2016   1  25  21.2             658
55    5  2016   1  29  36.8             645
58    5  2017   1  20  32.5             605
59    7  2013   1  13  44.9             373


,day,year,mo,da,dewp,NUM_COLLISIONS
count,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000
mean,3.998434,2015.999217,6.524070,15.723679,44.163170,599.109980
std,2.000391,2.000000,3.449676,8.801271,16.995303,100.277185
min,1.000000,2013.000000,1.000000,1.000000,-6.700000,188.000000
25%,2.000000,2014.000000,4.000000,8.000000,32.150000,531.000000
50%,4.000000,2016.000000,7.000000,16.000000,45.300000,602.000000
75%,6.000000,2018.000000,10.000000,23.000000,58.500000,665.000000
max,7.000000,2019.000000,12.000000,31.000000,74.100000,1161.000000


In [12]:
#Shuffle Data
shuffle = df_dewp.iloc[np.random.permutation(len(df_dewp))]
#Select Predictors
predictors = shuffle.iloc[:,0:-1]
print(predictors[:6])
print(shuffle[:6])

      day  year  mo  da  dewp
1421    4  2017   5  25  51.1
545     7  2015   2   1  10.2
1320    2  2013   5  21  55.5
2517    6  2017   9   2  45.2
691     4  2016   3  24  38.1
3340    1  2013  11  25   4.4
      day  year  mo  da  dewp  NUM_COLLISIONS
1421    4  2017   5  25  51.1             742
545     7  2015   2   1  10.2             361
1320    2  2013   5  21  55.5             697
2517    6  2017   9   2  45.2             562
691     4  2016   3  24  38.1             635
3340    1  2013  11  25   4.4             710


In [13]:
#Select last col as target
targets = shuffle.iloc[:,-1]

print(targets[:6])

1421    742
545     361
1320    697
2517    562
691     635
3340    710
Name: NUM_COLLISIONS, dtype: int64


In [14]:
#Split into test and train data
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
print(trainsize)
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize
print(testsize)
nppredictors = 5
noutputs = 1

2044
511


In [15]:
# logging for tensorflow
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

# removes a saved model from the last training attempt.
shutil.rmtree('/tmp/linear_regression_trained_model_dewp', ignore_errors=True)

#Setup Model
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_dewp', optimizer=tf.train.AdamOptimizer(learning_rate=0.00001), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))
print("starting to train");
#Train Model
estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_COLLISIONS, steps=10000)
preds = estimator.predict(x=predictors[trainsize:].values)
predslistscale = preds['scores']*SCALE_COLLISIONS

#test Model
pred = format(str(predslistscale))
rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('LinearRegression has RMSE of {0}'.format(rmse));

avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])
rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f69856ab7d0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_dewp', '_session_creation_timeout_secs': 7200}


starting to train


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/linear_regression_trained_model_dewp/model.ckpt.
INFO:tensorflow:loss = 0.2768998, step = 1
INFO:tensorflow:global_step/sec: 767.36
INFO:tensorflow:loss = 0.006016632, step = 101 (0.136 sec)
INFO:tensorflow:global_step/sec: 750.23
INFO:tensorflow:loss = 0.0055046105, step = 201 (0.134 sec)
INFO:tensorflow:global_step/sec: 861.733
INFO:tensorflow:loss = 0.0066261473, step = 301 (0.112 sec)
INFO:tensorflow:global_step/sec: 754.45
INFO:tensorflow:loss = 0.0068937447, step = 401 (0.132 sec)
INFO:tensorflow:global_step/sec: 882.746
INFO:tensorflow:loss = 0.008133821, step = 501 (0.118 sec)
INFO:tensorflow:global_step/sec: 878.639
INFO:tensorflow:loss = 0.008163027, step = 601 (0.115 sec)
INFO:tensorflow:global_step/sec: 779.254
INFO:tensorflow:loss = 0.007626701, step = 701 (0.12

LinearRegression has RMSE of 95.75721709818401
Just using average = 599.4006849315068 has RMSE of 101.27851174723051


In [16]:
print(len(predictors[trainsize:].values))
print(len(targets[trainsize:].values.reshape(testsize, noutputs)/SCALE_COLLISIONS))

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_dewp', enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors[trainsize:].values)))
preds = estimator.predict(x=predictors[trainsize:].values)

print(preds['scores'])
print(targets[trainsize:].values/SCALE_COLLISIONS)

testdf = pd.DataFrame.from_dict(data={
    'pred': preds['scores'],
    'actual': targets[trainsize:].values/SCALE_COLLISIONS
})

testdf['diff'] = testdf['actual'] - testdf['pred']

error=testdf['diff'].mean()*SCALE_COLLISIONS
avgcol=targets[trainsize:].mean()
print('The trained model has an aproximate error rate of {0} which equates to {1}%'.format(error, round((error/avgcol)*100),1));


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f6989698a90>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_dewp', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/linear_regression_trained_model_dew

511
511
[0.53421575 0.49107456 0.49575955 0.49392542 0.56014323 0.5377786
 0.5223685  0.53651047 0.48761994 0.5390038  0.49725395 0.4435207
 0.5142695  0.55664724 0.5556217  0.4559405  0.47730133 0.5024583
 0.48681208 0.4818581  0.55331534 0.48931074 0.4903469  0.45501387
 0.5434811  0.512435   0.53500646 0.5193931  0.49469087 0.49011216
 0.50368583 0.57227135 0.5332097  0.5539787  0.48971078 0.5274615
 0.5359507  0.48769894 0.48654956 0.44337437 0.47959533 0.5250092
 0.46282938 0.52377385 0.52091324 0.4930509  0.47698689 0.4867593
 0.56791437 0.48990378 0.50849086 0.46699712 0.47940958 0.47724116
 0.5077634  0.49327928 0.55553806 0.4620578  0.52435994 0.5629149
 0.5488726  0.4735266  0.5382052  0.51620543 0.49822935 0.5469527
 0.49283955 0.49834624 0.49440598 0.4798046  0.50224555 0.50002277
 0.5314395  0.4875895  0.5164654  0.47403085 0.5110501  0.49407378
 0.5435952  0.48025957 0.51183796 0.5238735  0.5174607  0.44084144
 0.50646    0.4979614  0.5576722  0.54823166 0.5386734  0.4983

The RMSE for the dewp model is slightly lower in comparison to the model produced for precipitation. As the RMSE is lower than the RMSE of the mean, it shows that the model has a higher level of accuracy in comparison to the using the mean.

##Visibility (visib)
A relationship was also uncovered between visibility and the number of collisions. This is a negative linear relationship where the visibility increases the number of collisions decrease. 

The process to produce the model follows the same process as above.

In [17]:
#Read Data
df_visib = pd.read_csv('https://raw.githubusercontent.com/matthew110395/12004210_DataAnalytics/main/coldata.csv', index_col=0, )
print(df_visib[:6])

   day  year  mo  da collision_date  NUM_COLLISIONS  temp  dewp     slp  \
1    5  2020   1  24     2020-01-24             524  37.3  33.7  1028.5   
2    2  2021   1  12     2021-01-12             278  37.0  29.1  1019.0   
3    5  2021   1  22     2021-01-22             254  36.5  28.4  1003.1   
4    3  2021   1  27     2021-01-27             262  34.6  33.8  1012.8   
5    2  2021   1  26     2021-01-26             263  31.9  23.4  1016.9   
6    1  2022   1  24     2022-01-24             237  34.5  23.8  1010.6   

   visib  ...   max   min  prcp   sndp  fog  rain_drizzle  snow_ice_pellets  \
1    6.5  ...  46.0  19.9  0.00  999.9    1             0                 0   
2   10.0  ...  44.1  21.0  0.00  999.9    0             0                 0   
3   10.0  ...  44.1  19.9  0.00  999.9    0             0                 0   
4    8.0  ...  41.0  28.9  0.25  999.9    1             1                 0   
5    9.0  ...  37.9  21.0  0.00  999.9    1             0                 0   


In [18]:
#Remove Cols not Required 
df_visib = df_visib.drop(columns=['collision_date', 'temp', 'prcp', 'slp','dewp','max','min','sndp','wdsp','mxpsd','gust','thunder','tornado_funnel_cloud','fog','rain_drizzle','snow_ice_pellets','hail'])
df_visib = df_visib.loc[df_visib["year"] != 2012]
df_visib = df_visib.loc[df_visib["year"] < 2020]
cols = df_visib['NUM_COLLISIONS']
df_visib = df_visib.drop(columns=['NUM_COLLISIONS'])
#Move target to end
df_visib.insert(loc=5, column='NUM_COLLISIONS', value=cols)
print(df_visib[:6])
df_visib.describe()

    day  year  mo  da  visib  NUM_COLLISIONS
49    4  2016   1  28   10.0             681
51    5  2014   1  17    6.7             589
54    1  2016   1  25   10.0             658
55    5  2016   1  29   10.0             645
58    5  2017   1  20   10.0             605
59    7  2013   1  13    4.3             373


,day,year,mo,da,visib,NUM_COLLISIONS
count,2556.000000,2556.0,2556.000000,2556.000000,2556.000000,2556.000000
mean,3.999218,2016.0,6.524257,15.725743,8.295618,599.118936
std,2.000391,2.0,3.449013,8.800168,2.207870,100.258581
min,1.000000,2013.0,1.000000,1.000000,0.200000,188.000000
25%,2.000000,2014.0,4.000000,8.000000,7.100000,531.000000
50%,4.000000,2016.0,7.000000,16.000000,9.400000,602.000000
75%,6.000000,2018.0,10.000000,23.000000,10.000000,665.000000
max,7.000000,2019.0,12.000000,31.000000,10.000000,1161.000000


In [19]:
#Shuffle the data
shuffle = df_visib.iloc[np.random.permutation(len(df_visib))]
predictors = shuffle.iloc[:,0:-1]
print(predictors[:6])
print(shuffle[:6])

      day  year  mo  da  visib
2696    2  2014   9  30    6.7
79      6  2019   1  19    9.8
3422    6  2015  12   5   10.0
3356    3  2014  11  26    6.7
3559    3  2013  12  11    9.9
2840    7  2016  10   2    4.4
      day  year  mo  da  visib  NUM_COLLISIONS
2696    2  2014   9  30    6.7             589
79      6  2019   1  19    9.8             479
3422    6  2015  12   5   10.0             629
3356    3  2014  11  26    6.7             742
3559    3  2013  12  11    9.9             605
2840    7  2016  10   2    4.4             486


In [20]:
# Select the last col as target
targets = shuffle.iloc[:,-1]

print(targets[:6])

2696    589
79      479
3422    629
3356    742
3559    605
2840    486
Name: NUM_COLLISIONS, dtype: int64


In [21]:
#Split to test and train data
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
print(trainsize)
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize
print(testsize)
nppredictors = 5
noutputs = 1

2044
512


In [22]:
# logging for tensorflow
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

# removes a saved model from the last training attempt.
shutil.rmtree('/tmp/linear_regression_trained_model_visib', ignore_errors=True)

#Setup Model
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_visib', optimizer=tf.train.AdamOptimizer(learning_rate=0.0001), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))
print("starting to train");
#train Model
estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_COLLISIONS, steps=10000)
preds = estimator.predict(x=predictors[trainsize:].values)
predslistscale = preds['scores']*SCALE_COLLISIONS
#Test Model
pred = format(str(predslistscale))
rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('LinearRegression has RMSE of {0}'.format(rmse));

avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])
rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f6989623290>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_visib', '_session_creation_timeout_secs': 7200}


starting to train


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/linear_regression_trained_model_visib/model.ckpt.
INFO:tensorflow:loss = 0.2742073, step = 1
INFO:tensorflow:global_step/sec: 719.248
INFO:tensorflow:loss = 0.0070698485, step = 101 (0.144 sec)
INFO:tensorflow:global_step/sec: 832.79
INFO:tensorflow:loss = 0.0074536, step = 201 (0.118 sec)
INFO:tensorflow:global_step/sec: 847.445
INFO:tensorflow:loss = 0.007559098, step = 301 (0.123 sec)
INFO:tensorflow:global_step/sec: 810.918
INFO:tensorflow:loss = 0.00721395, step = 401 (0.120 sec)
INFO:tensorflow:global_step/sec: 848.991
INFO:tensorflow:loss = 0.0056845434, step = 501 (0.118 sec)
INFO:tensorflow:global_step/sec: 814.265
INFO:tensorflow:loss = 0.007107227, step = 601 (0.123 sec)
INFO:tensorflow:global_step/sec: 904.801
INFO:tensorflow:loss = 0.0079799695, step = 701 (0.10

LinearRegression has RMSE of 95.02490044046145
Just using average = 600.2005870841488 has RMSE of 100.99356537020641


In [23]:
print(len(predictors[trainsize:].values))
print(len(targets[trainsize:].values.reshape(testsize, noutputs)/SCALE_COLLISIONS))

#testd = pd.DataFrame.from_records(predictors[trainsize:].values,columns=['day','year','month','da','prcp','fog','rain','snow','hail'])
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_visib', enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors[trainsize:].values)))
preds = estimator.predict(x=predictors[trainsize:].values)

print(preds['scores'])
print(targets[trainsize:].values/SCALE_COLLISIONS)

testdf = pd.DataFrame.from_dict(data={
    'pred': preds['scores'],
    'actual': targets[trainsize:].values/SCALE_COLLISIONS
})

testdf['diff'] = testdf['actual'] - testdf['pred']

error=testdf['diff'].mean()*SCALE_COLLISIONS
avgcol=targets[trainsize:].mean()
print('The trained model has an aproximate error rate of {0} which equates to {1}%'.format(error, round((error/avgcol)*100),1));


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f698966e610>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_visib', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/linear_regression_trained_model_vi

512
512
[0.4930513  0.48964942 0.52658933 0.55354035 0.48071313 0.46926457
 0.5534737  0.4741751  0.5334721  0.5372914  0.4900828  0.5430065
 0.55380553 0.44292232 0.48949558 0.5133777  0.5397377  0.55587566
 0.54923576 0.52314264 0.51653457 0.5000549  0.50659853 0.48151675
 0.46897262 0.5132227  0.4974867  0.46730012 0.51012564 0.49101174
 0.5452201  0.55843276 0.5268288  0.45996836 0.49134487 0.5584993
 0.47613788 0.50463563 0.48416352 0.49086615 0.473387   0.47687387
 0.4995968  0.4614625  0.48358625 0.51483506 0.5620351  0.5460287
 0.45416307 0.46849984 0.50902444 0.505555   0.48494315 0.4584965
 0.47581074 0.5041938  0.5553458  0.46635842 0.52336776 0.49803078
 0.56080973 0.51679945 0.5542693  0.5102459  0.53552014 0.5602274
 0.4648863  0.5065072  0.5001049  0.5005762  0.52159077 0.5018685
 0.5522017  0.47006923 0.49449837 0.4955241  0.5276064  0.5023916
 0.5328224  0.49324125 0.51787317 0.47525185 0.5009696  0.54844546
 0.5153221  0.508546   0.5347954  0.5349317  0.50299925 0.482

As the difference RMSE between the mean and the model is less, it is arguable that the model is not as accurate.

Although the RMSE value indicates a weaker model, the error rate is lower indicating that there is a significant error increasing the RMSE.

#Deep Learning Neural Network (DNN)
Although the primary outcome of assignment 1 was uncovering linear relationships, more complex relationships which cannot be predicted using a linear regressor were uncovered. In this case a Deep Learning Neural Network (DNN) can be used.

A Deep Learning Neural Network (DNN) is a form of unsupervised learning, where a number of hidden layers are used to uncover non-linear relationships. Karhunen, Raiko and Cho (2015) infer that deep learning neural networks work in a similar way to the human brain. This is where both the relationship between the input and output data is explored, as well as the relationship between the underlying data.

##Precipitation (prcp)
The process for training a DNN follows a similar process followed above for a Linear Regressor. The data cleansed and one hot encoded as part of assignment 1 is loaded from GitHub.

In [24]:
#Read Data
df = pd.read_csv('https://raw.githubusercontent.com/matthew110395/12004210_DataAnalytics/main/prcp_clean_dnn.csv', index_col=0, )
print(df[:6])

   year  da  NUM_COLLISIONS  temp  dewp     slp  visib  wdsp  mxpsd   gust  \
1  2020  24             524  37.3  33.7  1028.5    6.5   3.3    8.0  999.9   
2  2021  12             278  37.0  29.1  1019.0   10.0   6.5   12.0  999.9   
3  2021  22             254  36.5  28.4  1003.1   10.0   7.8   12.0   20.0   
4  2021  27             262  34.6  33.8  1012.8    8.0   7.8   12.0  999.9   
5  2021  26             263  31.9  23.4  1016.9    9.0   7.4   12.0  999.9   
6  2022  24             237  34.5  23.8  1010.6    9.7   7.4   12.0  999.9   

   ...  May  Nov  Oct  Sep  Mon  Sat  Sun  Thu  Tue  Wed  
1  ...    0    0    0    0    0    0    0    1    0    0  
2  ...    0    0    0    0    1    0    0    0    0    0  
3  ...    0    0    0    0    0    0    0    1    0    0  
4  ...    0    0    0    0    0    0    0    0    1    0  
5  ...    0    0    0    0    1    0    0    0    0    0  
6  ...    0    0    0    0    0    0    1    0    0    0  

[6 rows x 38 columns]


In [25]:
#Remove Cols not Required 
df_prcp_dnn = df.drop(columns=['temp', 'dewp', 'slp','visib','max','min','sndp','wdsp','mxpsd','gust','thunder','tornado_funnel_cloud'])
df_prcp_dnn = df_prcp_dnn.loc[df_prcp_dnn["year"] != 2012]
df_prcp_dnn = df_prcp_dnn.loc[df_prcp_dnn["year"] < 2020]
#Move target to the end
cols = df_prcp_dnn['NUM_COLLISIONS']
df_prcp_dnn = df_prcp_dnn.drop(columns=['NUM_COLLISIONS'])
df_prcp_dnn.insert(loc=25, column='NUM_COLLISIONS', value=cols)
print(df_prcp_dnn[:6])
df_prcp_dnn.describe()

    year  da  prcp  fog  rain_drizzle  snow_ice_pellets  hail  Apr  Aug  Dec  \
49  2016  28  0.09    0             0                 0     0    0    0    0   
51  2014  17  0.00    1             0                 0     0    0    0    0   
54  2016  25  0.02    0             0                 0     0    0    0    0   
55  2016  29  0.00    0             0                 0     0    0    0    0   
58  2017  20  0.00    0             0                 0     0    0    0    0   
59  2013  13  0.01    1             0                 0     0    0    0    0   

    ...  Nov  Oct  Sep  Mon  Sat  Sun  Thu  Tue  Wed  NUM_COLLISIONS  
49  ...    0    0    0    0    0    0    0    0    1             681  
51  ...    0    0    0    0    0    0    1    0    0             589  
54  ...    0    0    0    0    0    1    0    0    0             658  
55  ...    0    0    0    0    0    0    1    0    0             645  
58  ...    0    0    0    0    0    0    1    0    0             605  
59  ...    0 

,year,da,prcp,fog,rain_drizzle,snow_ice_pellets,hail,Apr,Aug,Dec,...,Nov,Oct,Sep,Mon,Sat,Sun,Thu,Tue,Wed,NUM_COLLISIONS
count,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,...,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000
mean,2015.989366,15.745569,0.122588,0.253249,0.375345,0.085467,0.000394,0.082316,0.083497,0.085467,...,0.082710,0.085467,0.079953,0.142970,0.143364,0.143757,0.142182,0.142576,0.142182,599.135093
std,1.996126,8.803199,0.329143,0.434958,0.484307,0.279630,0.019846,0.274899,0.276687,0.279630,...,0.275497,0.279630,0.271273,0.350111,0.350512,0.350913,0.349305,0.349709,0.349305,100.299164
min,2013.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,188.000000
25%,2014.000000,8.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,531.000000
50%,2016.000000,16.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,602.000000
75%,2018.000000,23.000000,0.060000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,665.000000
max,2019.000000,31.000000,3.760000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1161.000000


In [26]:
# Shuffle Data
shuffle = df_prcp_dnn.iloc[np.random.permutation(len(df_prcp_dnn))]
predictors = shuffle.iloc[:,0:-1]
print(predictors[:6])

      year  da  prcp  fog  rain_drizzle  snow_ice_pellets  hail  Apr  Aug  \
3019  2013   9  0.03    0             0                 0     0    0    0   
3561  2017  29  0.00    0             0                 1     0    0    0   
2027  2015  23  0.00    0             0                 0     0    0    0   
807   2015  23  0.00    0             0                 0     0    0    0   
506   2016   9  0.06    0             0                 1     0    0    0   
208   2015  17  0.00    0             0                 0     0    0    0   

      Dec  ...  May  Nov  Oct  Sep  Mon  Sat  Sun  Thu  Tue  Wed  
3019    0  ...    0    0    1    0    0    0    0    0    1    0  
3561    1  ...    0    0    0    0    0    0    0    1    0    0  
2027    0  ...    0    0    0    0    0    0    0    0    0    1  
807     0  ...    0    0    0    0    0    0    1    0    0    0  
506     0  ...    0    0    0    0    1    0    0    0    0    0  
208     0  ...    0    0    0    0    0    0    0    0    

In [27]:
#Select target as last col
targets = shuffle.iloc[:,-1]
print(targets[:6])

3019    529
3561    598
2027    623
807     586
506     571
208     495
Name: NUM_COLLISIONS, dtype: int64


In [28]:
#Split to test and train data
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize
noutputs = 1
# calculate the number of predictors
nppredictors = len(shuffle.columns) - noutputs
print(nppredictors)

25


The difference between the training of a DNN and linear regression model, is the addition of hidden layers. In order to optimise the trained model, the number of hidden layers, the number of nodes within each layer and the learning rate were all modified.

In [29]:
# removes a saved model from the last training attempt.
shutil.rmtree('/tmp/DNN_regression_trained_model_prcp', ignore_errors=True)

#Setup Model
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_prcp', hidden_units=[20,18,13], optimizer=tf.train.AdamOptimizer(learning_rate=0.01), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))
print("starting to train");
#Train Model
estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_COLLISIONS, steps=10000)

preds = estimator.predict(x=predictors[trainsize:].values)

predslistscale = preds['scores']*SCALE_COLLISIONS

#Test model
rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('DNNRegression has RMSE of {0}'.format(rmse));
avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])
rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f6985608850>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_prcp', '_session_creation_timeout_secs': 7200}


starting to train


Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/DNN_regression_trained_model_prcp/model.ckpt.
INFO:tensorflow:loss = 5797.1284, step = 1
INFO:tensorflow:global_step/sec: 484.233
INFO:tensorflow:loss = 0.093114644, step = 101 (0.209 sec)
INFO:tensorflow:global_step/sec: 633.315
INFO:tensorflow:loss = 0.072762355, step = 201 (0.160 sec)
INFO:tensorflow:global_step/sec: 600.974
INFO:tensorflow:loss = 0.059431594, step = 301 (0.169 sec)
INFO:tensorflow:global_step/sec: 581.296
INFO:tensorflow:loss = 0.04294809, step = 401 (0.171 sec)
INFO:tensorflow:global_step/sec: 614.568
INFO:tensorflow:loss = 0.030806527, step = 501 (0.162 sec)
INFO:tensorflow:global_step/sec: 601.412
INFO:tensorflow:loss = 0.027688656, step = 601 (0.166 sec)
INFO:tensorflow:global_ste

DNNRegression has RMSE of 77.11229799272591
Just using average = 599.723289020187 has RMSE of 98.31506286288999


In [30]:
print(len(predictors[trainsize:].values))
print(len(targets[trainsize:].values.reshape(testsize, noutputs)/SCALE_COLLISIONS))
#Ensure hidden layers match the model trained above
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_prcp', hidden_units=[20,18,13], enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors[trainsize:].values)))
preds = estimator.predict(x=predictors[trainsize:].values)

print(preds['scores'])
print(targets[trainsize:].values/SCALE_COLLISIONS)

testdf = pd.DataFrame.from_dict(data={
    'pred': preds['scores'],
    'actual': targets[trainsize:].values/SCALE_COLLISIONS
})

testdf['diff'] = testdf['actual'] - testdf['pred']

error=testdf['diff'].mean()*SCALE_COLLISIONS
avgcol=targets[trainsize:].mean()
print('The trained model has an aproximate error rate of {0} which equates to {1}%'.format(error, round((error/avgcol)*100),1));


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f69892dca10>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_prcp', '_session_creation_timeout_secs': 7200}


508
508


INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/DNN_regression_trained_model_prcp/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[0.5063163  0.54900837 0.5241063  0.4464979  0.41719916 0.3804229
 0.5627806  0.5216837  0.5723269  0.5345738  0.4760729  0.48885325
 0.46499184 0.46727848 0.56145835 0.562441   0.54951066 0.48909315
 0.5473968  0.5864883  0.55957127 0.58954227 0.59036994 0.5918138
 0.54280305 0.55159426 0.5442295  0.50568104 0.53995234 0.39843637
 0.58585703 0.5603374  0.5768122  0.5341629  0.49730456 0.5600103
 0.5656823  0.5426327  0.49772337 0.54801536 0.452973   0.57085854
 0.3779933  0.5636091  0.5153109  0.5246148  0.49161074 0.57500076
 0.5608868  0.5649049  0.5348468  0.4999415  0.5269826  0.46126568
 0.48901182 0.44556057 0.5270839  0.47129714 0.3719187  0.41603413
 0.42897987 0.5321904  0.55687034 0.4006964  0.49625728 0.5583056
 0.5591398  0.5199641  0.52011514 0.55865824 0.42880937 0.59242123
 0.4510392  0.53094155 0.48752865 0.5337252  0.588908   0.5142597
 0.51020217 0.5866565  0.41077453 0.53864133 0.52015203 0.5624
 0.56370866 0.5177094  0.5984142  0.53709704 0.5633903  0.544704
 0.475

The RMSE value is very similar to that of the linear regression model, indicating that both models are accurate predictors. The error rate of the DNN is higher, indicating the there is a higher number of errors, but the margin of error is lower.

##Dew Point (dewp)
As with the linear regressor the process of training each model follows a very similar process, with the number of hidden layers, the number of nodes within each layer and the learning rate changing dependant on the dataset.

In [74]:
#Read Data
df_dewp_dnn = pd.read_csv('https://raw.githubusercontent.com/matthew110395/12004210_DataAnalytics/main/dewp_clean_dnn.csv', index_col=0, )
print(df_dewp_dnn[:6])

   year  da  NUM_COLLISIONS  temp  dewp     slp  visib  wdsp  mxpsd   gust  \
1  2020  24             524  37.3  33.7  1028.5    6.5   3.3    8.0  999.9   
2  2021  12             278  37.0  29.1  1019.0   10.0   6.5   12.0  999.9   
3  2021  22             254  36.5  28.4  1003.1   10.0   7.8   12.0   20.0   
4  2021  27             262  34.6  33.8  1012.8    8.0   7.8   12.0  999.9   
5  2021  26             263  31.9  23.4  1016.9    9.0   7.4   12.0  999.9   
6  2022  24             237  34.5  23.8  1010.6    9.7   7.4   12.0  999.9   

   ...  May  Nov  Oct  Sep  Mon  Sat  Sun  Thu  Tue  Wed  
1  ...    0    0    0    0    0    0    0    1    0    0  
2  ...    0    0    0    0    1    0    0    0    0    0  
3  ...    0    0    0    0    0    0    0    1    0    0  
4  ...    0    0    0    0    0    0    0    0    1    0  
5  ...    0    0    0    0    1    0    0    0    0    0  
6  ...    0    0    0    0    0    0    1    0    0    0  

[6 rows x 38 columns]


In [75]:
#Remove Cols not Required 
df_dewp_dnn = df_dewp_dnn.drop(columns=['temp', 'prcp', 'slp','visib','max','min','sndp','wdsp','mxpsd','gust','thunder','tornado_funnel_cloud','fog','rain_drizzle','snow_ice_pellets','hail'])
df_dewp_dnn = df_dewp_dnn.loc[df_dewp_dnn["year"] != 2012]
df_dewp_dnn = df_dewp_dnn.loc[df_dewp_dnn["year"] < 2020]
#Move target to end
cols = df_dewp_dnn['NUM_COLLISIONS']
df_dewp_dnn = df_dewp_dnn.drop(columns=['NUM_COLLISIONS'])
df_dewp_dnn.insert(loc=21, column='NUM_COLLISIONS', value=cols)
print(df_dewp_dnn[:6])
df_dewp_dnn.describe()

    year  da  dewp  Apr  Aug  Dec  Feb  Jan  Jul  Jun  ...  Nov  Oct  Sep  \
49  2016  28  24.4    0    0    0    0    1    0    0  ...    0    0    0   
51  2014  17  35.8    0    0    0    0    1    0    0  ...    0    0    0   
54  2016  25  21.2    0    0    0    0    1    0    0  ...    0    0    0   
55  2016  29  36.8    0    0    0    0    1    0    0  ...    0    0    0   
58  2017  20  32.5    0    0    0    0    1    0    0  ...    0    0    0   
59  2013  13  44.9    0    0    0    0    1    0    0  ...    0    0    0   

    Mon  Sat  Sun  Thu  Tue  Wed  NUM_COLLISIONS  
49    0    0    0    0    0    1             681  
51    0    0    0    1    0    0             589  
54    0    0    1    0    0    0             658  
55    0    0    0    1    0    0             645  
58    0    0    0    1    0    0             605  
59    0    1    0    0    0    0             373  

[6 rows x 22 columns]


,year,da,dewp,Apr,Aug,Dec,Feb,Jan,Jul,Jun,...,Nov,Oct,Sep,Mon,Sat,Sun,Thu,Tue,Wed,NUM_COLLISIONS
count,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,...,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000
mean,2015.999217,15.723679,44.163170,0.082192,0.084932,0.084932,0.077104,0.084932,0.084540,0.082192,...,0.082192,0.084932,0.082192,0.143249,0.142857,0.142857,0.142857,0.142857,0.142857,599.109980
std,2.000000,8.801271,16.995303,0.274710,0.278834,0.278834,0.266808,0.278834,0.278251,0.274710,...,0.274710,0.278834,0.274710,0.350395,0.349996,0.349996,0.349996,0.349996,0.349996,100.277185
min,2013.000000,1.000000,-6.700000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,188.000000
25%,2014.000000,8.000000,32.150000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,531.000000
50%,2016.000000,16.000000,45.300000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,602.000000
75%,2018.000000,23.000000,58.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,665.000000
max,2019.000000,31.000000,74.100000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1161.000000


In [76]:
#Shuffle Data
shuffle = df_dewp_dnn.iloc[np.random.permutation(len(df_dewp_dnn))]

predictors = shuffle.iloc[:,0:-1]
print(predictors[:6])

      year  da  dewp  Apr  Aug  Dec  Feb  Jan  Jul  Jun  ...  May  Nov  Oct  \
1980  2019  30  71.1    0    0    0    0    0    1    0  ...    0    0    0   
2182  2014  10  61.4    0    1    0    0    0    0    0  ...    0    0    0   
525   2018  17  24.8    0    0    0    1    0    0    0  ...    0    0    0   
131   2019  17  15.0    0    0    0    0    1    0    0  ...    0    0    0   
1656  2017   1  53.8    0    0    0    0    0    0    1  ...    0    0    0   
306   2014   3  20.2    0    0    0    0    1    0    0  ...    0    0    0   

      Sep  Mon  Sat  Sun  Thu  Tue  Wed  
1980    0    1    0    0    0    0    0  
2182    0    0    1    0    0    0    0  
525     0    0    0    0    0    0    0  
131     0    0    0    0    0    0    1  
1656    0    0    0    0    0    0    1  
306     0    0    0    0    1    0    0  

[6 rows x 21 columns]


In [77]:
#Select last col as target
targets = shuffle.iloc[:,-1]
print(targets[:6])

1980    638
2182    512
525     604
131     586
1656    654
306     423
Name: NUM_COLLISIONS, dtype: int64


In [78]:
#Split to test and train data
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize

noutputs = 1
#Calculate number of outputs
nppredictors = len(shuffle.columns) - noutputs
print(nppredictors)

21


In [84]:
# removes a saved model from the last training attempt.
shutil.rmtree('/tmp/DNN_regression_trained_model_dewp', ignore_errors=True)

#Setup Model
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_dewp', hidden_units=[21,17,9], optimizer=tf.train.AdamOptimizer(learning_rate=0.0001), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

print("starting to train");

#Train model
estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_COLLISIONS, steps=10000)

preds = estimator.predict(x=predictors[trainsize:].values)
predslistscale = preds['scores']*SCALE_COLLISIONS
#Test Model
rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('DNNRegression has RMSE of {0}'.format(rmse));
avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])

rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f6984fd60d0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_dewp', '_session_creation_timeout_secs': 7200}


starting to train


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/DNN_regression_trained_model_dewp/model.ckpt.
INFO:tensorflow:loss = 2885.8882, step = 1
INFO:tensorflow:global_step/sec: 452.294
INFO:tensorflow:loss = 0.6163688, step = 101 (0.224 sec)
INFO:tensorflow:global_step/sec: 631.605
INFO:tensorflow:loss = 0.43207002, step = 201 (0.161 sec)
INFO:tensorflow:global_step/sec: 543.559
INFO:tensorflow:loss = 0.45611447, step = 301 (0.181 sec)
INFO:tensorflow:global_step/sec: 579.565
INFO:tensorflow:loss = 0.2883045, step = 401 (0.176 sec)
INFO:tensorflow:global_step/sec: 554.957
INFO:tensorflow:loss = 0.44561958, step = 501 (0.179 sec)
INFO:tensorflow:global_step/sec: 568.541
INFO:tensorflow:loss = 0.37659773, step = 601 (0.178 sec)
INFO:tensorflow:global_step/sec: 613.147
INFO:tensorflow:loss = 0.3675207, step = 701 (0.165 sec)
INFO:t

DNNRegression has RMSE of 96.59943561948128
Just using average = 598.2734833659491 has RMSE of 101.31319535932884


In [85]:
print(len(predictors[trainsize:].values))
print(len(targets[trainsize:].values.reshape(testsize, noutputs)/SCALE_COLLISIONS))

# Ensure number of hidden units match the trained model
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_dewp', hidden_units=[21,17,9], enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors[trainsize:].values)))
preds = estimator.predict(x=predictors[trainsize:].values)

print(preds['scores'])
print(targets[trainsize:].values/SCALE_COLLISIONS)

testdf = pd.DataFrame.from_dict(data={
    'pred': preds['scores'],
    'actual': targets[trainsize:].values/SCALE_COLLISIONS
})

testdf['diff'] = testdf['actual'] - testdf['pred']

error=testdf['diff'].mean()*SCALE_COLLISIONS
avgcol=targets[trainsize:].mean()
print('The trained model has an aproximate error rate of {0} which equates to {1}%'.format(error, round((error/avgcol)*100),1));


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f6984e5b810>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_dewp', '_session_creation_timeout_secs': 7200}


511
511


INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/DNN_regression_trained_model_dewp/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[0.4818532  0.51460123 0.31911603 0.45461652 0.512521   0.50714093
 0.4920572  0.53636235 0.50577813 0.47075906 0.47199425 0.5076965
 0.51449645 0.5595216  0.5128036  0.5276331  0.54849154 0.49425277
 0.48258165 0.49075547 0.5062065  0.4773247  0.5223157  0.48422638
 0.49502268 0.48024714 0.5085339  0.49631643 0.49778122 0.49048793
 0.39905182 0.567484   0.50202125 0.5045959  0.5104677  0.5532324
 0.49237752 0.47260496 0.49752718 0.50108325 0.452891   0.50552005
 0.5038974  0.5033033  0.4542569  0.3608052  0.49467486 0.50112534
 0.5621588  0.43251032 0.45935723 0.49433625 0.5368627  0.41053864
 0.5106945  0.43314728 0.5073223  0.44676185 0.5020465  0.47161767
 0.5382461  0.48666313 0.5174236  0.4902932  0.49441513 0.5227637
 0.4980762  0.4040285  0.33408618 0.47766992 0.4334986  0.47165924
 0.39555758 0.39627087 0.5014044  0.4531174  0.4120762  0.5423504
 0.51209754 0.49326056 0.46020252 0.48183224 0.5203091  0.38524652
 0.5550864  0.50632936 0.47430155 0.52269053 0.526151   0.55424213

As shown by the RMSE value, this model is a more efficient way to predict the number of collisions in comparison to using the mean. In comparison to the linear regression model trained, the RMSE is lower indicating the DNN makes more accurate predictions. As with the DNN for precipitation, the RMSE is lower than the linear model the error rate is higher indicating there is more errors but the margin of error is lower.

##Sea Level Pressure(slp)
Through the analysis carried out in assignment 1, no clear relationship between sea level pressure and the number of collisions was uncovered. A DNN will be used to attempt to predict the number of collisions at a given pressure point.

In [38]:
#Read Data
df_slp_dnn = pd.read_csv('https://raw.githubusercontent.com/matthew110395/12004210_DataAnalytics/main/slp_clean_dnn.csv', index_col=0, )
print(df[:6])

   year  da  NUM_COLLISIONS  temp  dewp     slp  visib  wdsp  mxpsd   gust  \
1  2020  24             524  37.3  33.7  1028.5    6.5   3.3    8.0  999.9   
2  2021  12             278  37.0  29.1  1019.0   10.0   6.5   12.0  999.9   
3  2021  22             254  36.5  28.4  1003.1   10.0   7.8   12.0   20.0   
4  2021  27             262  34.6  33.8  1012.8    8.0   7.8   12.0  999.9   
5  2021  26             263  31.9  23.4  1016.9    9.0   7.4   12.0  999.9   
6  2022  24             237  34.5  23.8  1010.6    9.7   7.4   12.0  999.9   

   ...  May  Nov  Oct  Sep  Mon  Sat  Sun  Thu  Tue  Wed  
1  ...    0    0    0    0    0    0    0    1    0    0  
2  ...    0    0    0    0    1    0    0    0    0    0  
3  ...    0    0    0    0    0    0    0    1    0    0  
4  ...    0    0    0    0    0    0    0    0    1    0  
5  ...    0    0    0    0    1    0    0    0    0    0  
6  ...    0    0    0    0    0    0    1    0    0    0  

[6 rows x 38 columns]


In [39]:
#Remove Cols not Required 
df_slp_dnn = df_slp_dnn.drop(columns=['temp', 'prcp', 'dewp','visib','max','min','sndp','wdsp','mxpsd','gust','thunder','tornado_funnel_cloud','fog','rain_drizzle','snow_ice_pellets','hail'])
df_slp_dnn = df_slp_dnn.loc[df_slp_dnn["year"] != 2012]
df_slp_dnn = df_slp_dnn.loc[df_slp_dnn["year"] < 2020]
#Move target to the end
cols = df_slp_dnn['NUM_COLLISIONS']
df_slp_dnn = df_slp_dnn.drop(columns=['NUM_COLLISIONS'])
df_slp_dnn.insert(loc=21, column='NUM_COLLISIONS', value=cols)
print(df_slp_dnn[:6])
df_slp_dnn.describe()

    year  da     slp  Apr  Aug  Dec  Feb  Jan  Jul  Jun  ...  Nov  Oct  Sep  \
49  2016  28  1016.1    0    0    0    0    1    0    0  ...    0    0    0   
51  2014  17  1014.8    0    0    0    0    1    0    0  ...    0    0    0   
54  2016  25  1021.4    0    0    0    0    1    0    0  ...    0    0    0   
55  2016  29   999.4    0    0    0    0    1    0    0  ...    0    0    0   
58  2017  20  1015.5    0    0    0    0    1    0    0  ...    0    0    0   
59  2013  13  1020.7    0    0    0    0    1    0    0  ...    0    0    0   

    Mon  Sat  Sun  Thu  Tue  Wed  NUM_COLLISIONS  
49    0    0    0    0    0    1             681  
51    0    0    0    1    0    0             589  
54    0    0    1    0    0    0             658  
55    0    0    0    1    0    0             645  
58    0    0    0    1    0    0             605  
59    0    1    0    0    0    0             373  

[6 rows x 22 columns]


,year,da,slp,Apr,Aug,Dec,Feb,Jan,Jul,Jun,...,Nov,Oct,Sep,Mon,Sat,Sun,Thu,Tue,Wed,NUM_COLLISIONS
count,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,...,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000
mean,2016.000391,15.719765,1016.777221,0.082192,0.084932,0.084540,0.077104,0.084932,0.084932,0.082192,...,0.082192,0.084932,0.082192,0.143249,0.142857,0.142857,0.142857,0.142857,0.142466,599.147162
std,2.000294,8.796698,7.628429,0.274710,0.278834,0.278251,0.266808,0.278834,0.278834,0.274710,...,0.274710,0.278834,0.274710,0.350395,0.349996,0.349996,0.349996,0.349996,0.349596,100.268048
min,2013.000000,1.000000,989.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,188.000000
25%,2014.000000,8.000000,1012.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,531.000000
50%,2016.000000,16.000000,1016.700000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,602.000000
75%,2018.000000,23.000000,1021.700000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,665.000000
max,2019.000000,31.000000,1044.200000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1161.000000


In [40]:
#Shuffle dataset
shuffle = df_slp_dnn.iloc[np.random.permutation(len(df_slp_dnn))]

predictors = shuffle.iloc[:,0:-1]
print(predictors[:6])

      year  da     slp  Apr  Aug  Dec  Feb  Jan  Jul  Jun  ...  May  Nov  Oct  \
2443  2016  17  1014.4    0    1    0    0    0    0    0  ...    0    0    0   
655   2014  11  1002.9    0    0    0    0    0    0    0  ...    0    0    0   
87    2016  15  1014.8    0    0    0    0    1    0    0  ...    0    0    0   
1625  2018   9  1018.7    0    0    0    0    0    0    1  ...    0    0    0   
1182  2016   5  1019.3    1    0    0    0    0    0    0  ...    0    0    0   
1434  2013  14  1015.8    0    0    0    0    0    0    0  ...    1    0    0   

      Sep  Mon  Sat  Sun  Thu  Tue  Wed  
2443    0    0    0    0    0    1    0  
655     0    1    0    0    0    0    0  
87      0    0    0    0    1    0    0  
1625    0    0    0    0    0    0    0  
1182    0    1    0    0    0    0    0  
1434    0    1    0    0    0    0    0  

[6 rows x 21 columns]


In [41]:
#Select Target
targets = shuffle.iloc[:,-1]

# print out the first 6 rows of the targets data.
print(targets[:6])

2443    658
655     510
87      722
1625    706
1182    672
1434    565
Name: NUM_COLLISIONS, dtype: int64


In [42]:
#Split into test and train data
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize

noutputs = 1
#Calculate the number of predictors
nppredictors = len(shuffle.columns) - noutputs
print(nppredictors)

21


In [43]:
# removes a saved model from the last training attempt.
shutil.rmtree('/tmp/DNN_regression_trained_model_slp', ignore_errors=True)
#Setup model
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_slp', hidden_units=[19,15,11], optimizer=tf.train.AdamOptimizer(learning_rate=0.01), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

print("starting to train");

# Train model
estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_COLLISIONS, steps=10000)

preds = estimator.predict(x=predictors[trainsize:].values)
predslistscale = preds['scores']*SCALE_COLLISIONS
#Test Model
rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('DNNRegression has RMSE of {0}'.format(rmse));

avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])
rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f698953fb10>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_slp', '_session_creation_timeout_secs': 7200}


starting to train


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/DNN_regression_trained_model_slp/model.ckpt.
INFO:tensorflow:loss = 9936.371, step = 1
INFO:tensorflow:global_step/sec: 464.988
INFO:tensorflow:loss = 1.244927, step = 101 (0.219 sec)
INFO:tensorflow:global_step/sec: 614.381
INFO:tensorflow:loss = 0.36806673, step = 201 (0.164 sec)
INFO:tensorflow:global_step/sec: 575.612
INFO:tensorflow:loss = 0.11767705, step = 301 (0.173 sec)
INFO:tensorflow:global_step/sec: 583.415
INFO:tensorflow:loss = 0.031092502, step = 401 (0.172 sec)
INFO:tensorflow:global_step/sec: 592.056
INFO:tensorflow:loss = 0.01956394, step = 501 (0.169 sec)
INFO:tensorflow:global_step/sec: 587.81
INFO:tensorflow:loss = 0.01150495, step = 601 (0.169 sec)
INFO:tensorflow:global_step/sec: 583.142
INFO:tensorflow:loss = 0.015598632, step = 701 (0.175 sec)
INFO:t

DNNRegression has RMSE of 79.70311085693434
Just using average = 598.7788649706458 has RMSE of 98.45554082087295


In [44]:
print(len(predictors[trainsize:].values))
print(len(targets[trainsize:].values.reshape(testsize, noutputs)/SCALE_COLLISIONS))

#Ensure hidden units match the trained model
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_slp', hidden_units=[19,15,11], enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors[trainsize:].values)))
preds = estimator.predict(x=predictors[trainsize:].values)

print(preds['scores'])
print(targets[trainsize:].values/SCALE_COLLISIONS)

testdf = pd.DataFrame.from_dict(data={
    'pred': preds['scores'],
    'actual': targets[trainsize:].values/SCALE_COLLISIONS
})

testdf['diff'] = testdf['actual'] - testdf['pred']

error=testdf['diff'].mean()*SCALE_COLLISIONS
avgcol=targets[trainsize:].mean()
print('The trained model has an aproximate error rate of {0} which equates to {1}%'.format(error, round((error/avgcol)*100),1));


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f698571cc90>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_slp', '_session_creation_timeout_secs': 7200}


511
511


INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/DNN_regression_trained_model_slp/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[0.54682606 0.5556914  0.5329282  0.521293   0.55673933 0.5291626
 0.5033573  0.53117627 0.45112374 0.5126334  0.45786372 0.5461756
 0.5115131  0.58452356 0.55373687 0.537376   0.5745998  0.4934062
 0.5258286  0.58835816 0.56955415 0.5131042  0.5125766  0.5403559
 0.44633624 0.5540217  0.56178236 0.441593   0.57178134 0.4244301
 0.5319812  0.5297657  0.5033051  0.50895125 0.5455781  0.55492324
 0.5583681  0.56263155 0.58475876 0.5472646  0.619179   0.5005402
 0.5434918  0.429134   0.4075162  0.5589485  0.50904137 0.3876207
 0.51352364 0.5555644  0.5077263  0.5411454  0.53805083 0.50603956
 0.578005   0.57618684 0.54920924 0.5644084  0.49727663 0.43904588
 0.53781617 0.550493   0.5566992  0.46487942 0.51258147 0.52088666
 0.5262752  0.596177   0.43721464 0.50537044 0.55371463 0.5162443
 0.53503406 0.5887415  0.4353313  0.42495155 0.55601346 0.47680178
 0.52018225 0.5546715  0.58957124 0.55052394 0.51438355 0.51643646
 0.57406884 0.4989576  0.47414768 0.5258431  0.49450502 0.5760749
 0.5

Although no linear relationship was uncovered, it can be argued there is a relationship present due to the RMSE value which is lower than the mean value. This relationship is also shown in the error rate which is comparative to the other models produced.

##Gust
As with sea level pressure, no linear relationship between the maximum gust and the number of collisions was uncovered.

In [45]:
#Read data
df_gust_dnn = pd.read_csv('https://raw.githubusercontent.com/matthew110395/12004210_DataAnalytics/main/gust_clean_dnn.csv', index_col=0, )
print(df_gust_dnn[:6])

    year  da  NUM_COLLISIONS  temp  dewp     slp  visib  wdsp  mxpsd  gust  \
3   2021  22             254  36.5  28.4  1003.1   10.0   7.8   12.0  20.0   
11  2020  15             508  43.9  38.3  1019.4    8.2   5.4   14.0  15.0   
12  2021   1             257  39.6  29.3  1029.3   10.0   7.6   14.0  20.0   
14  2022  25             235  41.6  31.8  1013.2   10.0   9.6   15.0  19.0   
18  2021   3             186  41.1  32.3  1018.0   10.0  10.3   19.0  27.0   
19  2020   2             413  39.6  28.9  1011.8   10.0  13.0   19.0  26.0   

    ...  May  Nov  Oct  Sep  Mon  Sat  Sun  Thu  Tue  Wed  
3   ...    0    0    0    0    0    0    0    1    0    0  
11  ...    0    0    0    0    0    0    0    0    1    0  
12  ...    0    0    0    0    0    0    0    1    0    0  
14  ...    0    0    0    0    1    0    0    0    0    0  
18  ...    0    0    0    0    0    1    0    0    0    0  
19  ...    0    0    0    0    0    0    0    0    0    1  

[6 rows x 38 columns]


In [46]:
#Remove Cols not Required 
df_gust_dnn = df_gust_dnn.drop(columns=['temp', 'prcp', 'dewp','visib','max','min','sndp','wdsp','mxpsd','slp','thunder','tornado_funnel_cloud','fog','rain_drizzle','snow_ice_pellets','hail'])
df_gust_dnn = df_gust_dnn.loc[df_gust_dnn["year"] != 2012]
df_gust_dnn = df_gust_dnn.loc[df_gust_dnn["year"] < 2020]
#Move target col to end
cols = df_gust_dnn['NUM_COLLISIONS']
df_gust_dnn = df_gust_dnn.drop(columns=['NUM_COLLISIONS'])
df_gust_dnn.insert(loc=21, column='NUM_COLLISIONS', value=cols)
print(df_gust_dnn[:6])
df_gust_dnn.describe()

    year  da  gust  Apr  Aug  Dec  Feb  Jan  Jul  Jun  ...  Nov  Oct  Sep  \
74  2016  17  18.1    0    0    0    0    1    0    0  ...    0    0    0   
76  2014   9  20.0    0    0    0    0    1    0    0  ...    0    0    0   
79  2019  19  21.0    0    0    0    0    1    0    0  ...    0    0    0   
80  2015  11  17.1    0    0    0    0    1    0    0  ...    0    0    0   
83  2015  29  20.0    0    0    0    0    1    0    0  ...    0    0    0   
85  2019  13  15.9    0    0    0    0    1    0    0  ...    0    0    0   

    Mon  Sat  Sun  Thu  Tue  Wed  NUM_COLLISIONS  
74    0    1    0    0    0    0             451  
76    0    0    0    0    0    1             561  
79    0    0    0    0    0    0             479  
80    0    1    0    0    0    0             341  
83    0    0    0    0    0    1             519  
85    0    1    0    0    0    0             374  

[6 rows x 22 columns]


,year,da,gust,Apr,Aug,Dec,Feb,Jan,Jul,Jun,...,Nov,Oct,Sep,Mon,Sat,Sun,Thu,Tue,Wed,NUM_COLLISIONS
count,1629.00000,1629.000000,1629.000000,1629.000000,1629.000000,1629.000000,1629.000000,1629.000000,1629.000000,1629.000000,...,1629.000000,1629.000000,1629.000000,1629.000000,1629.000000,1629.000000,1629.000000,1629.000000,1629.000000,1629.000000
mean,2015.91283,15.702885,27.511602,0.095764,0.042357,0.104359,0.095150,0.108656,0.046041,0.061387,...,0.096378,0.087784,0.071209,0.139963,0.141191,0.139963,0.151627,0.138122,0.145488,596.513198
std,2.01341,8.667634,7.366770,0.294358,0.201465,0.305819,0.293513,0.311302,0.209637,0.240113,...,0.295200,0.283067,0.257253,0.347055,0.348325,0.347055,0.358769,0.345133,0.352700,104.479660
min,2013.00000,1.000000,14.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,188.000000
25%,2014.00000,8.000000,22.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,526.000000
50%,2016.00000,16.000000,26.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,597.000000
75%,2018.00000,23.000000,31.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,663.000000
max,2019.00000,31.000000,71.100000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1161.000000


In [47]:
# Shuffle Data
shuffle = df_gust_dnn.iloc[np.random.permutation(len(df_gust_dnn))]

predictors = shuffle.iloc[:,0:-1]
print(predictors[:6])

      year  da  gust  Apr  Aug  Dec  Feb  Jan  Jul  Jun  ...  May  Nov  Oct  \
106   2017  29  22.0    0    0    0    0    1    0    0  ...    0    0    0   
3631  2016  17  32.1    0    0    1    0    0    0    0  ...    0    0    0   
795   2015  14  28.0    0    0    0    0    0    0    0  ...    0    0    0   
1436  2019   8  22.0    0    0    0    0    0    0    0  ...    1    0    0   
1110  2019  21  28.0    1    0    0    0    0    0    0  ...    0    0    0   
1129  2017   8  28.9    1    0    0    0    0    0    0  ...    0    0    0   

      Sep  Mon  Sat  Sun  Thu  Tue  Wed  
106     0    0    1    0    0    0    0  
3631    0    0    0    0    0    0    0  
795     0    0    0    0    0    0    0  
1436    0    0    0    0    0    1    0  
1110    0    0    1    0    0    0    0  
1129    0    0    0    0    0    0    0  

[6 rows x 21 columns]


In [48]:
# Select last col as a target
targets = shuffle.iloc[:,-1]
print(targets[:6])

106     428
3631    623
795     507
1436    649
1110    398
1129    627
Name: NUM_COLLISIONS, dtype: int64


In [49]:
# Split into test and train data
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize

noutputs = 1
# Calculate the number of predictors
nppredictors = len(shuffle.columns) - noutputs
print(nppredictors)

21


In [50]:
# removes a saved model from the last training attempt.
shutil.rmtree('/tmp/DNN_regression_trained_model_gust', ignore_errors=True)

#Setup model
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_gust', hidden_units=[23,19,11], optimizer=tf.train.AdamOptimizer(learning_rate=0.001), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

print("starting to train");

# Train the model
estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_COLLISIONS, steps=10000)

preds = estimator.predict(x=predictors[trainsize:].values)
predslistscale = preds['scores']*SCALE_COLLISIONS

#Test the model
rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('DNNRegression has RMSE of {0}'.format(rmse));
avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])
rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f69857ab410>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_gust', '_session_creation_timeout_secs': 7200}


starting to train


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/DNN_regression_trained_model_gust/model.ckpt.
INFO:tensorflow:loss = 80768.34, step = 1
INFO:tensorflow:global_step/sec: 462.411
INFO:tensorflow:loss = 0.14845058, step = 101 (0.219 sec)
INFO:tensorflow:global_step/sec: 665.979
INFO:tensorflow:loss = 0.013256403, step = 201 (0.153 sec)
INFO:tensorflow:global_step/sec: 670.743
INFO:tensorflow:loss = 0.010560701, step = 301 (0.149 sec)
INFO:tensorflow:global_step/sec: 630.141
INFO:tensorflow:loss = 0.011680349, step = 401 (0.158 sec)
INFO:tensorflow:global_step/sec: 623.015
INFO:tensorflow:loss = 0.011477644, step = 501 (0.161 sec)
INFO:tensorflow:global_step/sec: 612.758
INFO:tensorflow:loss = 0.009742392, step = 601 (0.163 sec)
INFO:tensorflow:global_step/sec: 562.415
INFO:tensorflow:loss = 0.012344847, step = 701 (0.178 sec

DNNRegression has RMSE of 91.9706605886673
Just using average = 596.3514965464313 has RMSE of 103.55154233421264


In [51]:
print(len(predictors[trainsize:].values))
print(len(targets[trainsize:].values.reshape(testsize, noutputs)/SCALE_COLLISIONS))

#Ensure the hidden units match the trained model
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_gust', hidden_units=[23,19,11], enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors[trainsize:].values)))
preds = estimator.predict(x=predictors[trainsize:].values)

print(preds['scores'])
print(targets[trainsize:].values/SCALE_COLLISIONS)

testdf = pd.DataFrame.from_dict(data={
    'pred': preds['scores'],
    'actual': targets[trainsize:].values/SCALE_COLLISIONS
})

testdf['diff'] = testdf['actual'] - testdf['pred']

error=testdf['diff'].mean()*SCALE_COLLISIONS
avgcol=targets[trainsize:].mean()
print('The trained model has an aproximate error rate of {0} which equates to {1}%'.format(error, round((error/avgcol)*100),1));


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f6988f14110>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_gust', '_session_creation_timeout_secs': 7200}


326
326


INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/DNN_regression_trained_model_gust/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[0.5070013  0.5516282  0.54840076 0.48284504 0.5207979  0.4892027
 0.5289301  0.45238695 0.5339557  0.54814994 0.5378042  0.5288331
 0.4753481  0.5448718  0.5343888  0.48902097 0.5220012  0.476482
 0.5201158  0.5258397  0.5349783  0.52133596 0.53429604 0.44876382
 0.49566755 0.517786   0.5320708  0.5278579  0.4761928  0.48006633
 0.48181656 0.5433951  0.47660324 0.4932333  0.48243734 0.52154386
 0.5300238  0.504466   0.5196644  0.47444794 0.44301754 0.4863446
 0.5309394  0.4940553  0.52281845 0.5372747  0.519419   0.50254434
 0.51200944 0.5301151  0.49654493 0.48933753 0.50977385 0.4938307
 0.47847065 0.518213   0.51581264 0.5212314  0.4857166  0.54374003
 0.472257   0.4842331  0.47952327 0.526529   0.48776135 0.54902506
 0.48996368 0.5085628  0.52305144 0.47757563 0.4970931  0.54709417
 0.5188292  0.489786   0.5137935  0.5151523  0.5020509  0.5507229
 0.487788   0.54538196 0.5262925  0.52487224 0.5264717  0.45700836
 0.52436674 0.49740377 0.48234782 0.5311031  0.4758903  0.5149297
 0.

This model shows there is no relationship between gust and the number of collisions.

The RMSE value is over 4 times the mean value indicating the model is unable to make accurate predictions. This is also reflected in the very high error rate.

##Maximum Sustained Wind Speed (mxpsd)
In an attempt to predict the number of collisions given the maximum sustained wind speed a DNN will be used as no linear relationship was uncovered within assignment 1.

In [52]:
#Read the data
df_mxpsd_dnn = pd.read_csv('https://raw.githubusercontent.com/matthew110395/12004210_DataAnalytics/main/mxpsd_clean_dnn.csv', index_col=0, )
print(df_mxpsd_dnn[:6])

   year  da  NUM_COLLISIONS  temp  dewp     slp  visib  wdsp  mxpsd   gust  \
1  2020  24             524  37.3  33.7  1028.5    6.5   3.3    8.0  999.9   
2  2021  12             278  37.0  29.1  1019.0   10.0   6.5   12.0  999.9   
3  2021  22             254  36.5  28.4  1003.1   10.0   7.8   12.0   20.0   
4  2021  27             262  34.6  33.8  1012.8    8.0   7.8   12.0  999.9   
5  2021  26             263  31.9  23.4  1016.9    9.0   7.4   12.0  999.9   
6  2022  24             237  34.5  23.8  1010.6    9.7   7.4   12.0  999.9   

   ...  May  Nov  Oct  Sep  Mon  Sat  Sun  Thu  Tue  Wed  
1  ...    0    0    0    0    0    0    0    1    0    0  
2  ...    0    0    0    0    1    0    0    0    0    0  
3  ...    0    0    0    0    0    0    0    1    0    0  
4  ...    0    0    0    0    0    0    0    0    1    0  
5  ...    0    0    0    0    1    0    0    0    0    0  
6  ...    0    0    0    0    0    0    1    0    0    0  

[6 rows x 38 columns]


In [53]:
#Remove the cols not required
df_mxpsd_dnn = df_mxpsd_dnn.drop(columns=['temp', 'prcp', 'dewp','visib','max','min','sndp','wdsp','gust','slp','thunder','tornado_funnel_cloud','fog','rain_drizzle','snow_ice_pellets','hail'])
df_mxpsd_dnn = df_mxpsd_dnn.loc[df_mxpsd_dnn["year"] != 2012]
df_mxpsd_dnn = df_mxpsd_dnn.loc[df_mxpsd_dnn["year"] < 2020]
#Move the target to the end
cols = df_mxpsd_dnn['NUM_COLLISIONS']
df_mxpsd_dnn = df_mxpsd_dnn.drop(columns=['NUM_COLLISIONS'])
df_mxpsd_dnn.insert(loc=21, column='NUM_COLLISIONS', value=cols)
print(df_mxpsd_dnn[:6])
df_mxpsd_dnn.describe()

    year  da  mxpsd  Apr  Aug  Dec  Feb  Jan  Jul  Jun  ...  Nov  Oct  Sep  \
49  2016  28    8.9    0    0    0    0    1    0    0  ...    0    0    0   
51  2014  17    8.9    0    0    0    0    1    0    0  ...    0    0    0   
54  2016  25    8.9    0    0    0    0    1    0    0  ...    0    0    0   
55  2016  29    9.9    0    0    0    0    1    0    0  ...    0    0    0   
58  2017  20    9.9    0    0    0    0    1    0    0  ...    0    0    0   
59  2013  13    9.9    0    0    0    0    1    0    0  ...    0    0    0   

    Mon  Sat  Sun  Thu  Tue  Wed  NUM_COLLISIONS  
49    0    0    0    0    0    1             681  
51    0    0    0    1    0    0             589  
54    0    0    1    0    0    0             658  
55    0    0    0    1    0    0             645  
58    0    0    0    1    0    0             605  
59    0    1    0    0    0    0             373  

[6 rows x 22 columns]


,year,da,mxpsd,Apr,Aug,Dec,Feb,Jan,Jul,Jun,...,Nov,Oct,Sep,Mon,Sat,Sun,Thu,Tue,Wed,NUM_COLLISIONS
count,2553.000000,2553.000000,2553.000000,2553.000000,2553.000000,2553.000000,2553.000000,2553.000000,2553.000000,2553.000000,...,2553.000000,2553.000000,2553.000000,2553.000000,2553.000000,2553.000000,2553.000000,2553.000000,2553.000000,2553.000000
mean,2016.001567,15.737172,17.240110,0.082256,0.084998,0.084998,0.077164,0.084998,0.084998,0.082256,...,0.081864,0.084998,0.081473,0.143361,0.142969,0.142969,0.142969,0.142577,0.142186,599.033686
std,2.000587,8.797367,5.858333,0.274808,0.278933,0.278933,0.266904,0.278933,0.278933,0.274808,...,0.274212,0.278933,0.273613,0.350509,0.350110,0.350110,0.350110,0.349710,0.349309,100.284761
min,2013.000000,1.000000,5.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,188.000000
25%,2014.000000,8.000000,13.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,531.000000
50%,2016.000000,16.000000,15.900000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,602.000000
75%,2018.000000,23.000000,20.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,665.000000
max,2019.000000,31.000000,49.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1161.000000


In [54]:
# shuffle the data
shuffle = df_mxpsd_dnn.iloc[np.random.permutation(len(df_mxpsd_dnn))]

predictors = shuffle.iloc[:,0:-1]
print(predictors[:6])

      year  da  mxpsd  Apr  Aug  Dec  Feb  Jan  Jul  Jun  ...  May  Nov  Oct  \
1116  2017  29   21.0    1    0    0    0    0    0    0  ...    0    0    0   
187   2014  23   19.0    0    0    0    0    1    0    0  ...    0    0    0   
1407  2017  12   17.1    0    0    0    0    0    0    0  ...    1    0    0   
2282  2014  27   12.0    0    1    0    0    0    0    0  ...    0    0    0   
2321  2017  18   13.0    0    1    0    0    0    0    0  ...    0    0    0   
3359  2018  10   32.1    0    0    0    0    0    0    0  ...    0    1    0   

      Sep  Mon  Sat  Sun  Thu  Tue  Wed  
1116    0    0    0    0    0    0    0  
187     0    0    0    0    0    0    1  
1407    0    0    0    0    1    0    0  
2282    0    0    0    0    0    1    0  
2321    0    0    0    0    1    0    0  
3359    0    0    0    0    0    0    0  

[6 rows x 21 columns]


In [55]:
# Select the target as the last col
targets = shuffle.iloc[:,-1]
print(targets[:6])

1116    661
187     612
1407    817
2282    538
2321    612
3359    606
Name: NUM_COLLISIONS, dtype: int64


In [56]:
#Split into test and train data
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize

noutputs = 1
# Calculate the number of predictors
nppredictors = len(shuffle.columns) - noutputs
print(nppredictors)

21


In [57]:
# removes a saved model from the last training attempt.
shutil.rmtree('/tmp/DNN_regression_trained_model_mxpsd', ignore_errors=True)

#Setup the model
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_mxpsd', hidden_units=[17,13,7], optimizer=tf.train.AdamOptimizer(learning_rate=0.001), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

print("starting to train");

# Train the model
estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_COLLISIONS, steps=10000)

preds = estimator.predict(x=predictors[trainsize:].values)
predslistscale = preds['scores']*SCALE_COLLISIONS

# Test the model
rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('DNNRegression has RMSE of {0}'.format(rmse));
avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])
rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f698b847e50>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_mxpsd', '_session_creation_timeout_secs': 7200}


starting to train


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/DNN_regression_trained_model_mxpsd/model.ckpt.
INFO:tensorflow:loss = 7281.42, step = 1
INFO:tensorflow:global_step/sec: 470.374
INFO:tensorflow:loss = 0.32891485, step = 101 (0.218 sec)
INFO:tensorflow:global_step/sec: 616.899
INFO:tensorflow:loss = 0.2024329, step = 201 (0.159 sec)
INFO:tensorflow:global_step/sec: 575.004
INFO:tensorflow:loss = 0.09926839, step = 301 (0.177 sec)
INFO:tensorflow:global_step/sec: 572.791
INFO:tensorflow:loss = 0.16393805, step = 401 (0.172 sec)
INFO:tensorflow:global_step/sec: 584.382
INFO:tensorflow:loss = 0.14116764, step = 501 (0.174 sec)
INFO:tensorflow:global_step/sec: 612.36
INFO:tensorflow:loss = 0.13836263, step = 601 (0.163 sec)
INFO:tensorflow:global_step/sec: 603.732
INFO:tensorflow:loss = 0.1368792, step = 701 (0.165 sec)
INFO:te

DNNRegression has RMSE of 92.59173675913111
Just using average = 599.0034280117532 has RMSE of 98.76589237104777


In [58]:
print(len(predictors[trainsize:].values))
print(len(targets[trainsize:].values.reshape(testsize, noutputs)/SCALE_COLLISIONS))

# Ensure the hidden units match the trained model
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_mxpsd', hidden_units=[17,13,7], enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors[trainsize:].values)))
preds = estimator.predict(x=predictors[trainsize:].values)

print(preds['scores'])
print(targets[trainsize:].values/SCALE_COLLISIONS)

testdf = pd.DataFrame.from_dict(data={
    'pred': preds['scores'],
    'actual': targets[trainsize:].values/SCALE_COLLISIONS
})

testdf['diff'] = testdf['actual'] - testdf['pred']

error=testdf['diff'].mean()*SCALE_COLLISIONS
avgcol=targets[trainsize:].mean()
print('The trained model has an aproximate error rate of {0} which equates to {1}%'.format(error, round((error/avgcol)*100),1));


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f69895ce110>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_mxpsd', '_session_creation_timeout_secs': 7200}


511
511


INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/DNN_regression_trained_model_mxpsd/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[0.6197088  0.61292255 0.45971116 0.562188   0.5749753  0.627877
 0.5536151  0.5251169  0.4734179  0.52584887 0.56128913 0.60150003
 0.6132801  0.43617    0.5261491  0.6402318  0.65793586 0.4947307
 0.45797455 0.596325   0.577111   0.5916977  0.5878715  0.6005496
 0.5269165  0.5387591  0.61788464 0.62090635 0.587026   0.58937526
 0.48686    0.59996444 0.5799774  0.66578424 0.6190591  0.4180082
 0.5760781  0.5900259  0.6349804  0.5417632  0.5991643  0.575372
 0.5377871  0.5978111  0.6126435  0.60970896 0.6198152  0.58549166
 0.55941916 0.5315228  0.4642753  0.62802327 0.48908383 0.46225625
 0.46850598 0.5347105  0.5813071  0.6387332  0.5002119  0.46419138
 0.5395841  0.5191006  0.5754804  0.51653206 0.6023753  0.579029
 0.5869737  0.5156487  0.61550295 0.5507567  0.5576838  0.47351295
 0.5469168  0.5206457  0.61585724 0.5760814  0.5976771  0.52899027
 0.5580237  0.52622396 0.5708613  0.64892906 0.51670724 0.568297
 0.48287004 0.54368407 0.5934662  0.5935642  0.45962042 0.54199564
 0.676

The low RMSE value suggests that a relationship between the maximum sustained wind speed and the number of collisions exist. The model produced can be used to predict the number of collisions with a degree of accuracy. As with the other DNN models produced the error rate is higher.

##Whole Dataset
As the purpose of this assignment is to accurately predict the number of collisions given the weather condition**s**, all available weather conditions are used as input variables to train a DNN model.

In [86]:
#Read the data
df = pd.read_csv('https://raw.githubusercontent.com/matthew110395/12004210_DataAnalytics/main/datadnn.csv', index_col=0, )
print(df[:6])

   year  da  NUM_COLLISIONS  temp  dewp     slp  visib  wdsp  mxpsd   gust  \
1  2020  24             524  37.3  33.7  1028.5    6.5   3.3    8.0  999.9   
2  2021  12             278  37.0  29.1  1019.0   10.0   6.5   12.0  999.9   
3  2021  22             254  36.5  28.4  1003.1   10.0   7.8   12.0   20.0   
4  2021  27             262  34.6  33.8  1012.8    8.0   7.8   12.0  999.9   
5  2021  26             263  31.9  23.4  1016.9    9.0   7.4   12.0  999.9   
6  2022  24             237  34.5  23.8  1010.6    9.7   7.4   12.0  999.9   

   ...  May  Nov  Oct  Sep  Mon  Sat  Sun  Thu  Tue  Wed  
1  ...    0    0    0    0    0    0    0    1    0    0  
2  ...    0    0    0    0    1    0    0    0    0    0  
3  ...    0    0    0    0    0    0    0    1    0    0  
4  ...    0    0    0    0    0    0    0    0    1    0  
5  ...    0    0    0    0    1    0    0    0    0    0  
6  ...    0    0    0    0    0    0    1    0    0    0  

[6 rows x 38 columns]


As the whole dataset contains the error values for Dew Point, Sea Level Pressure, Maximum Sustained Wind Speed and Gust; these must be removed.

In [87]:
#Clean the data
dnn = df.loc[df["year"] != 2012]
dnn = dnn.loc[dnn["year"] < 2020]
dnn = dnn.loc[dnn["dewp"] != 9999]
dnn = dnn.loc[dnn["slp"] != 9999]
dnn = dnn.loc[dnn["mxpsd"] != 999.9]
dnn = dnn.loc[dnn["gust"] != 999.9]
#Move the target to the end
cols = dnn['NUM_COLLISIONS']
dnn = dnn.drop(columns=['NUM_COLLISIONS'])
dnn.insert(loc=37, column='NUM_COLLISIONS', value=cols)
print(dnn[:6])
dnn.describe()

    year  da  temp  dewp     slp  visib  wdsp  mxpsd  gust   max  ...  Nov  \
74  2016  17  40.2  32.3  1007.3    9.2   7.7   12.0  18.1  51.1  ...    0   
76  2014   9  23.5   8.3  1034.2   10.0   7.9   12.0  20.0  28.9  ...    0   
79  2019  19  34.5  29.7  1022.0    9.8   6.9   13.0  21.0  39.9  ...    0   
80  2015  11  27.1  12.1  1035.5   10.0   8.8   13.0  17.1  37.0  ...    0   
83  2015  29  29.2  20.9  1022.9   10.0   8.5   13.0  20.0  36.0  ...    0   
85  2019  13  26.0  12.8  1030.5   10.0   8.0   13.0  15.9  30.9  ...    0   

    Oct  Sep  Mon  Sat  Sun  Thu  Tue  Wed  NUM_COLLISIONS  
74    0    0    0    1    0    0    0    0             451  
76    0    0    0    0    0    0    0    1             561  
79    0    0    0    0    0    0    0    0             479  
80    0    0    0    1    0    0    0    0             341  
83    0    0    0    0    0    0    0    1             519  
85    0    0    0    1    0    0    0    0             374  

[6 rows x 38 columns]


,year,da,temp,dewp,slp,visib,wdsp,mxpsd,gust,max,...,Nov,Oct,Sep,Mon,Sat,Sun,Thu,Tue,Wed,NUM_COLLISIONS
count,1629.00000,1629.000000,1629.000000,1629.000000,1629.000000,1629.000000,1629.000000,1629.000000,1629.000000,1629.00000,...,1629.000000,1629.000000,1629.000000,1629.000000,1629.000000,1629.000000,1629.000000,1629.000000,1629.000000,1629.000000
mean,2015.91283,15.702885,47.909638,45.903254,1015.632904,8.225599,12.602087,20.060896,27.511602,55.73407,...,0.096378,0.087784,0.071209,0.139963,0.141191,0.139963,0.151627,0.138122,0.145488,596.513198
std,2.01341,8.667634,13.746339,247.352840,8.134237,2.227285,3.986056,5.294117,7.366770,13.52726,...,0.295200,0.283067,0.257253,0.347055,0.348325,0.347055,0.358769,0.345133,0.352700,104.479660
min,2013.00000,1.000000,5.800000,-6.700000,989.500000,0.600000,4.500000,8.900000,14.000000,18.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,188.000000
25%,2014.00000,8.000000,38.100000,28.200000,1010.600000,7.000000,10.000000,15.900000,22.000000,46.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,526.000000
50%,2016.00000,16.000000,47.000000,40.200000,1015.400000,9.300000,12.000000,19.000000,26.000000,55.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,597.000000
75%,2018.00000,23.000000,58.800000,52.800000,1021.100000,10.000000,14.400000,22.900000,31.100000,66.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,663.000000
max,2019.00000,31.000000,77.500000,9999.900000,1039.100000,10.000000,39.300000,49.000000,71.100000,87.10000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1161.000000


In [89]:
# Shuffle the data
shuffle = dnn.iloc[np.random.permutation(len(dnn))]
predictors = shuffle.iloc[:,0:-1]
print(predictors[:6])

      year  da  temp  dewp     slp  visib  wdsp  mxpsd  gust   max  ...  May  \
2924  2015  24  51.5  38.3  1027.1   10.0   9.4   17.1  26.0  63.0  ...    0   
389   2014   7  28.1  15.7  1023.2   10.0   9.0   14.0  19.0  32.0  ...    0   
1800  2017  16  59.8  52.9  1018.9    9.9  15.5   22.0  32.1  66.0  ...    0   
1318  2015  21  54.8  41.1  1014.1   10.0   7.7   14.0  19.0  64.9  ...    1   
2639  2014  13  59.9  53.2  1022.2    9.6   7.6   15.0  21.0  70.0  ...    0   
3005  2017  10  68.8  65.6  1015.0    5.9  11.0   21.0  28.9  77.0  ...    0   

      Nov  Oct  Sep  Mon  Sat  Sun  Thu  Tue  Wed  
2924    0    1    0    0    0    0    0    0    0  
389     0    0    0    0    0    0    1    0    0  
1800    0    0    0    0    0    0    1    0    0  
1318    0    0    0    0    0    0    0    0    1  
2639    0    0    1    0    0    0    0    0    0  
3005    0    1    0    1    0    0    0    0    0  

[6 rows x 37 columns]


In [90]:
#select the target as the last col
targets = shuffle.iloc[:,-1]
print(targets[:6])

2924    519
389     675
1800    657
1318    615
2639    558
3005    768
Name: NUM_COLLISIONS, dtype: int64


In [91]:
# Split into test and train data
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize

noutputs = 1
# Calculate the number of predictors
nppredictors = len(shuffle.columns) - noutputs
print(nppredictors)


37


In [93]:
# removes a saved model from the last training attempt.
shutil.rmtree('/tmp/DNN_regression_trained_model', ignore_errors=True)

#Setup the model
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model', hidden_units=[17,9,5,3], optimizer=tf.train.AdamOptimizer(learning_rate=0.001), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

print("starting to train");

# Train the model
estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_COLLISIONS, steps=10000)

preds = estimator.predict(x=predictors[trainsize:].values)
predslistscale = preds['scores']*SCALE_COLLISIONS
#Test the model
rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('DNNRegression has RMSE of {0}'.format(rmse));
avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])
rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f69856419d0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model', '_session_creation_timeout_secs': 7200}


starting to train


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/DNN_regression_trained_model/model.ckpt.
INFO:tensorflow:loss = 78.1907, step = 1
INFO:tensorflow:global_step/sec: 478.049
INFO:tensorflow:loss = 0.38407242, step = 101 (0.214 sec)
INFO:tensorflow:global_step/sec: 596.525
INFO:tensorflow:loss = 0.13543814, step = 201 (0.167 sec)
INFO:tensorflow:global_step/sec: 656.951
INFO:tensorflow:loss = 0.012446139, step = 301 (0.155 sec)
INFO:tensorflow:global_step/sec: 630.194
INFO:tensorflow:loss = 0.0113518, step = 401 (0.158 sec)
INFO:tensorflow:global_step/sec: 563.676
INFO:tensorflow:loss = 0.008229288, step = 501 (0.176 sec)
INFO:tensorflow:global_step/sec: 591.682
INFO:tensorflow:loss = 0.0087835565, step = 601 (0.168 sec)
INFO:tensorflow:global_step/sec: 648.349
INFO:tensorflow:loss = 0.0086564915, step = 701 (0.156 sec)
INFO:

DNNRegression has RMSE of 84.60673873963464
Just using average = 596.55871066769 has RMSE of 104.57564089372552


In [94]:
print(predictors[trainsize:].values)

#Ensure the hidden units match the trained model
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model', hidden_units=[17,9,5,3], enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors[trainsize:].values)))
preds = estimator.predict(x=predictors[trainsize:].values)

print(preds['scores'])
print(targets[trainsize:].values/SCALE_COLLISIONS)

testdf = pd.DataFrame.from_dict(data={
    'pred': preds['scores'],
    'actual': targets[trainsize:].values/SCALE_COLLISIONS
})

testdf['diff'] = testdf['actual'] - testdf['pred']

error=testdf['diff'].mean()*SCALE_COLLISIONS
avgcol=targets[trainsize:].mean()
print('The trained model has an aproximate error rate of {0} which equates to {1}%'.format(error, round((error/avgcol)*100),1));


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f69893db4d0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model', '_session_creation_timeout_secs': 7200}


[[2.018e+03 5.000e+00 4.750e+01 ... 0.000e+00 0.000e+00 0.000e+00]
 [2.013e+03 1.900e+01 3.590e+01 ... 0.000e+00 0.000e+00 0.000e+00]
 [2.016e+03 1.400e+01 5.800e+00 ... 0.000e+00 0.000e+00 0.000e+00]
 ...
 [2.013e+03 2.600e+01 5.020e+01 ... 0.000e+00 0.000e+00 0.000e+00]
 [2.014e+03 1.000e+01 6.940e+01 ... 0.000e+00 0.000e+00 1.000e+00]
 [2.018e+03 1.500e+01 4.610e+01 ... 0.000e+00 0.000e+00 0.000e+00]]


INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/DNN_regression_trained_model/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[0.4960399  0.4456477  0.39058387 0.5475331  0.54891425 0.5210385
 0.5318333  0.5486218  0.41578734 0.38593492 0.59302896 0.50176954
 0.5217861  0.46751556 0.5236506  0.54228014 0.5886998  0.56803644
 0.56422985 0.4773662  0.5200675  0.55904335 0.59088856 0.54373586
 0.56062186 0.5190677  0.56101006 0.5286563  0.5246736  0.53569335
 0.5160985  0.48941115 0.49363893 0.5549494  0.5891978  0.5680552
 0.45636275 0.58715254 0.47624832 0.568396   0.5348954  0.5475349
 0.41068873 0.4827219  0.49168992 0.5611435  0.397279   0.5111623
 0.5135496  0.46726406 0.549048   0.42037567 0.48997295 0.58580214
 0.52456826 0.5401323  0.5751318  0.53076124 0.47012526 0.49686134
 0.44280255 0.40975973 0.4380358  0.5734564  0.53073186 0.53820854
 0.5045014  0.5365789  0.45125043 0.5774837  0.47968945 0.5253165
 0.5349813  0.51444614 0.532233   0.59344107 0.48791525 0.42087618
 0.60464257 0.4592435  0.5429901  0.53882694 0.54601926 0.51798946
 0.6072702  0.4912275  0.5297626  0.520552   0.57103544 0.5064908
 

The low RMSE and the low percentage error rate suggest that using the whole cleaned dataset is an accurate predictor of the number of collisions.

## Location
As identified in assignment 1, as the location tends towards the centre of New York there is stronger linear relationships. This suggests there is a link between the number of collisions, location and the observed weather conditions. A DNN will be trained to attempt to predict the number of collisions given the location, day and weather conditions.

In [95]:
#Read the data and extract from the zip
#Reference - (geeksforgeeks.org 2021)
df_loc = pd.read_csv('https://raw.githubusercontent.com/matthew110395/12004210_DataAnalytics/main/locdnn.zip', index_col=0,compression='zip' )
print(df_loc[:6])

   year  da  NUM_COLLISIONS   latitude  longitude  temp  dewp     slp  visib  \
1  2018   2               1  40.681750 -73.967480  14.7   2.0  1024.9   10.0   
2  2018   2               1  40.645370 -73.945110  14.7   2.0  1024.9   10.0   
3  2018   2               1  40.614830 -73.998380  14.7   2.0  1024.9   10.0   
4  2018   2               1  40.592190 -74.087395  14.7   2.0  1024.9   10.0   
5  2018   2               1  40.769817 -73.782370  14.7   2.0  1024.9   10.0   
6  2018   2               1  40.660175 -73.928200  14.7   2.0  1024.9   10.0   

   wdsp  ...  May  Nov  Oct  Sep  Mon  Sat  Sun  Thu  Tue  Wed  
1  12.9  ...    0    0    0    0    1    0    0    0    0    0  
2  12.9  ...    0    0    0    0    1    0    0    0    0    0  
3  12.9  ...    0    0    0    0    1    0    0    0    0    0  
4  12.9  ...    0    0    0    0    1    0    0    0    0    0  
5  12.9  ...    0    0    0    0    1    0    0    0    0    0  
6  12.9  ...    0    0    0    0    1    0    0  

In [67]:
#Remove unrequired cols
df_loc_dnn = df_loc.drop(columns=['thunder','tornado_funnel_cloud','fog','rain_drizzle','snow_ice_pellets','hail'])
#Clean data
df_loc_dnn = df_loc_dnn.loc[df_loc_dnn["year"] != 2012]
df_loc_dnn = df_loc_dnn.loc[df_loc_dnn["year"] < 2020]
df_loc_dnn = df_loc_dnn.loc[df_loc_dnn["dewp"] != 9999]
df_loc_dnn = df_loc_dnn.loc[df_loc_dnn["slp"] != 9999]
df_loc_dnn = df_loc_dnn.loc[df_loc_dnn["mxpsd"] != 999.9]
df_loc_dnn = df_loc_dnn.loc[df_loc_dnn["gust"] != 999.9]
#Move the target to the end
cols = df_loc_dnn['NUM_COLLISIONS']
df_loc_dnn = df_loc_dnn.drop(columns=['NUM_COLLISIONS'])
df_loc_dnn.insert(loc=33, column='NUM_COLLISIONS', value=cols)
print(df_loc_dnn[:6])
df_loc_dnn.describe()

   year  da   latitude  longitude  temp  dewp     slp  visib  wdsp  mxpsd  \
1  2018   2  40.681750 -73.967480  14.7   2.0  1024.9   10.0  12.9   20.0   
2  2018   2  40.645370 -73.945110  14.7   2.0  1024.9   10.0  12.9   20.0   
3  2018   2  40.614830 -73.998380  14.7   2.0  1024.9   10.0  12.9   20.0   
4  2018   2  40.592190 -74.087395  14.7   2.0  1024.9   10.0  12.9   20.0   
5  2018   2  40.769817 -73.782370  14.7   2.0  1024.9   10.0  12.9   20.0   
6  2018   2  40.660175 -73.928200  14.7   2.0  1024.9   10.0  12.9   20.0   

   ...  Nov  Oct  Sep  Mon  Sat  Sun  Thu  Tue  Wed  NUM_COLLISIONS  
1  ...    0    0    0    1    0    0    0    0    0               1  
2  ...    0    0    0    1    0    0    0    0    0               1  
3  ...    0    0    0    1    0    0    0    0    0               1  
4  ...    0    0    0    1    0    0    0    0    0               1  
5  ...    0    0    0    1    0    0    0    0    0               1  
6  ...    0    0    0    1    0    0    

,year,da,latitude,longitude,temp,dewp,slp,visib,wdsp,mxpsd,...,Nov,Oct,Sep,Mon,Sat,Sun,Thu,Tue,Wed,NUM_COLLISIONS
count,830297.000000,830297.000000,830297.000000,830297.000000,830297.000000,830297.000000,830297.000000,830297.000000,830297.000000,830297.000000,...,830297.000000,830297.000000,830297.000000,830297.000000,830297.000000,830297.000000,830297.000000,830297.000000,830297.000000,830297.000000
mean,2016.070154,15.638823,40.723907,-73.920916,48.419836,47.274900,1015.556656,8.199143,12.598551,20.021545,...,0.102479,0.093866,0.074335,0.146763,0.114698,0.142687,0.170187,0.141897,0.150735,1.027090
std,1.991298,8.613159,0.078454,0.086634,13.750834,261.636367,8.136580,2.230079,3.921832,5.219745,...,0.303277,0.291643,0.262315,0.353870,0.318657,0.349754,0.375797,0.348945,0.357791,0.180994
min,2013.000000,1.000000,40.498949,-74.253006,5.800000,-6.700000,989.500000,0.600000,4.500000,8.900000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,2014.000000,8.000000,40.668860,-73.976715,38.400000,28.800000,1010.700000,6.900000,10.000000,15.900000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,2016.000000,16.000000,40.722470,-73.929210,47.800000,41.300000,1015.300000,9.300000,12.000000,19.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,2018.000000,23.000000,40.768165,-73.866650,59.500000,53.400000,1021.000000,10.000000,14.400000,22.900000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,2019.000000,31.000000,40.912884,-73.663010,77.500000,9999.900000,1039.100000,10.000000,39.300000,49.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,11.000000


In [68]:
#Set the scale to the maximum value
SCALE_LOC=11
# Shuffle data
shuffle = df_loc_dnn.iloc[np.random.permutation(len(df_loc_dnn))]

predictors = shuffle.iloc[:,0:-1]
print(predictors[:6])

         year  da   latitude  longitude  temp  dewp     slp  visib  wdsp  \
857719   2015  20  40.783248 -73.944723  58.2  45.6  1022.8    9.7  21.3   
1212490  2016  15  40.777092 -73.982110  64.5  53.4  1022.0   10.0  14.4   
462091   2019  16  40.759575 -73.777030  39.7  24.5  1026.2   10.0  18.4   
418980   2019  28  40.787390 -73.938225  65.4  61.6  1015.1    9.2   7.7   
673714   2016  21  40.699680 -73.798640  64.6  62.1  1010.7    6.0  12.9   
195999   2016  17  40.732348 -73.984939  41.5  35.8  1011.9    6.2  12.0   

         mxpsd  ...  May  Nov  Oct  Sep  Mon  Sat  Sun  Thu  Tue  Wed  
857719    26.0  ...    0    0    1    0    1    0    0    0    0    0  
1212490   22.9  ...    0    0    0    1    0    0    0    0    0    1  
462091    28.0  ...    0    1    0    0    0    0    0    0    0    0  
418980    20.0  ...    0    0    0    0    0    0    0    0    1    0  
673714    18.1  ...    0    0    1    0    0    0    0    1    0    0  
195999    17.1  ...    0    0    0 

In [69]:
# Select last col as target
targets = shuffle.iloc[:,-1]
print(targets[:6])

857719     2
1212490    1
462091     1
418980     1
673714     1
195999     1
Name: NUM_COLLISIONS, dtype: int64


In [70]:
# Split into test and train data
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize

noutputs = 1
# Calculate the number of predictors
nppredictors = len(shuffle.columns) - noutputs
print(nppredictors)

33


In [71]:
# removes a saved model from the last training attempt.
shutil.rmtree('/tmp/DNN_regression_trained_model_loc', ignore_errors=True)

#Setup the model
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_loc', hidden_units=[19,15,11,7], optimizer=tf.train.AdamOptimizer(learning_rate=0.1), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

print("starting to train");

# Train the model.
estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_LOC, steps=10000)

preds = estimator.predict(x=predictors[trainsize:].values)
predslistscale = preds['scores']*SCALE_LOC
#Test Model
rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('DNNRegression has RMSE of {0}'.format(rmse));
avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])
rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f69856e5250>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_loc', '_session_creation_timeout_secs': 7200}


starting to train


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/DNN_regression_trained_model_loc/model.ckpt.
INFO:tensorflow:loss = 39883.492, step = 1
INFO:tensorflow:global_step/sec: 428.494
INFO:tensorflow:loss = 0.030600436, step = 101 (0.239 sec)
INFO:tensorflow:global_step/sec: 536.126
INFO:tensorflow:loss = 0.0028584637, step = 201 (0.187 sec)
INFO:tensorflow:global_step/sec: 542.49
INFO:tensorflow:loss = 0.00018860513, step = 301 (0.181 sec)
INFO:tensorflow:global_step/sec: 563.579
INFO:tensorflow:loss = 7.167452e-05, step = 401 (0.177 sec)
INFO:tensorflow:global_step/sec: 580.306
INFO:tensorflow:loss = 0.00012837196, step = 501 (0.173 sec)
INFO:tensorflow:global_step/sec: 517.867
INFO:tensorflow:loss = 0.0002505085, step = 601 (0.193 sec)
INFO:tensorflow:global_step/sec: 562.261
INFO:tensorflow:loss = 0.00025046951, step = 701 (

DNNRegression has RMSE of 0.18250732634269354
Just using average = 1.0270219816119848 has RMSE of 0.18125579252895552


In [73]:
print(predictors[trainsize:].values)
#Ensure the number of hidden units match the trained model
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_loc', hidden_units=[19,15,11,7], enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors[trainsize:].values)))
preds = estimator.predict(x=predictors[trainsize:].values)

print(preds['scores'])
print(targets[trainsize:].values/SCALE_COLLISIONS)

testdf = pd.DataFrame.from_dict(data={
    'pred': preds['scores'],
    'actual': targets[trainsize:].values/SCALE_LOC
})

testdf['diff'] = testdf['actual'] - testdf['pred']

error=testdf['diff'].mean()*SCALE_COLLISIONS
avgcol=targets[trainsize:].mean()
print('The trained model has an aproximate error rate of {0} which equates to {1}%'.format(error, round((error/avgcol)*100),1));


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f698953ccd0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_loc', '_session_creation_timeout_secs': 7200}


[[2.01300000e+03 7.00000000e+00 4.07984710e+01 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.01300000e+03 5.00000000e+00 4.07529363e+01 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.01800000e+03 1.20000000e+01 4.06017340e+01 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [2.01400000e+03 9.00000000e+00 4.06777741e+01 ... 0.00000000e+00
  0.00000000e+00 1.00000000e+00]
 [2.01700000e+03 7.00000000e+00 4.08226170e+01 ... 0.00000000e+00
  1.00000000e+00 0.00000000e+00]
 [2.01700000e+03 1.40000000e+01 4.08964960e+01 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]


INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/DNN_regression_trained_model_loc/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[0.09145673 0.09145673 0.09145673 ... 0.09145673 0.09145673 0.09145673]
[0.00086133 0.00086133 0.00086133 ... 0.00086133 0.00086133 0.00086133]
The trained model has an aproximate error rate of 2.2522994060669705 which equates to 219%


Despite extensive training, the model is not accurately able to predict the number of collisions. This is inferred by the RMSE which is close to that of the mean RMSE. 

#Conclusion
As shown above the given weather conditions for a particular day within New York, the number of collisions can be accurately predicted.

Arguments can be made for both models; the linear regression models appear to be less efficient due to the higher RMSE values in comparison to the DNN models. In contrast the error rate is lower for the linear regression models. This suggests that although the DNN models produce more errors, but the margin of error is lower, due to the RMSE placing a larger weighting on larger errors.

In hindsight the way location has been encoded does not allow for accurate predictions to be made as the number of collisions always tends towards 1 for a given location.

It is clear that the models produced above can accurately predict the number of collisions as set out in the specification of the assignment.


#References
geeksforgeeks.org (2021) Read a zipped file as a Pandas DataFrame [online]. Available from <<https://www.geeksforgeeks.org/read-a-zipped-file-as-a-pandas-dataframe/>? [12 November 2022] 

IBM (n.d.) What is linear regression? [online]. Available from <<https://www.ibm.com/uk-en/topics/linear-regression>> [17 November 2022] 

Karhunen, J., Raiko, T. and Cho, K. (2015) 'Chapter 7 - Unsupervised deep learning: A short review.' In Advances in Independent Component Analysis and Learning Machines. Academic Press. Ch. 7. 135-142.

Zhang, Z. (2019) Understand Data Normalization in Machine Learning [online]. Available from <<https://towardsdatascience.com/understand-data-normalization-in-machine-learning-8ff3062101f0>> [17 November 2022] 

Zulkifli, H. (2018) Understanding Learning Rates and How It Improves Performance in Deep Learning [online]. Available from <<https://towardsdatascience.com/understanding-learning-rates-and-how-it-improves-performance-in-deep-learning-d0d4059c1c10>> [17 November 2022] 

